In [0]:

from pyspark.sql.functions import expr, date_trunc, date_sub, date_add, lit, udf, col, avg, desc,  concat, lit, coalesce, current_date, round,  datediff, to_date, mean, min, max, mode, median
from pyspark.sql.types import FloatType
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from plotly import graph_objects as go
from pyspark.sql import functions as F
import pandas as pd
import plotly.graph_objects as go

In [0]:
#dbutils.widgets.removeAll()

In [0]:
import datetime
from dateutil.parser import parse

# Get the current date
current_date = datetime.datetime.now().date()

# Calculate the default start date as 2023-05-01
default_start_date = "2023-05-01"

# Create the end date filter widget with the current date as the default
dbutils.widgets.text("End Date", str(current_date))

# Retrieve the selected end date from the widget
end_date_string = dbutils.widgets.get("End Date")
end_date = parse(end_date_string).date()

# Create the start date filter widget with the default value
dbutils.widgets.text("Start Date", default_start_date)

# Retrieve the selected start date from the widget
start_date_string = dbutils.widgets.get("Start Date")
start_date = parse(start_date_string).date()


In [0]:
# Create a dictionary to map clinic names to IDs
clinic_mapping = {
    "All Clinics": "",
    "Clinic B": "83a53614-f7cc-4ebf-a117-3beceda02747",
    "Clinic N": "fff0ed73-282e-4f22-a7d9-e9396aa0af99",
    "Clinic U": "62ca07e4-b021-4324-a4ca-35f95831c281",
    "Clinic C": "152aa029-a643-4ee8-b5c6-246294603453"
}

clinic_names = list(clinic_mapping.keys())
dbutils.widgets.dropdown("Clinic", clinic_names[0], clinic_names)
selected_clinic_name = dbutils.widgets.get("Clinic")
clinic = clinic_mapping[selected_clinic_name]




In [0]:
from datetime import datetime, timedelta
end_date_last_7_days = datetime.now().date()
start_date_last_7_days = end_date_last_7_days - timedelta(days=6)

In [0]:
#dbutils.widgets.removeAll()

### Definitions and Calculations
- Referred count: Number of patient referred via 'clinician-patient-referred' event (Referral form).
- Activated count: Number of patient who completed 'Welcome' section of VideoAsk.
- Activated: (Activated count / Referred count) * 100%.
- Completed DA: Number of patients who completed decision aid.
- Meaningfully Engaged: (Number of patients who are activated, and clicks/opens at least 75% of the SMS or email / Total activated count) * 100%.
- AvgDaysBecomeActivated: Date difference between the referral date and activation date.
- Program progress: (The number of lessons and journeys completed / total number of lessons and journeys) * 100%.
- DAProgress: (The number of decision aid sections completed / total number of decision aid sections ) * 100%.

In [0]:
# 

In [0]:
# filter by program?
# filter by ckd stage?
# filter by month?
# before and after
# % that changed (treatment preference to modality selection)
# % transplant, before and after
# preference confidence and CKD understanding before and after?

In [0]:
# referral list
# maybe set up a filter by date in the future

# conditional statement for all clinics
if selected_clinic_name == "All Clinics":
    clinic_condition = ""  # No filter
else:
    clinic_condition = f"AND p.clinicId = '{clinic}'"

referred = spark.sql(f"""
    select 
    distinct 
    e.payload.clientId as sourceId,
    --first(e.payload.sourceId) as clinicianId,
    p.clinicId,
    min(date(e.timestamp)) as ReferralDate
    from prodeu.events e 
    LEFT JOIN prodeu.patients p on e.payload.clientId = p.clientId  
    where e.type = 'clinician-patient-referred'
    --where e.type = 'patient-referred'
    --where e.type = 'videoask-completed' or e.type = 'clinician-patient-referred'
    AND e.timestamp >= '{start_date}' 
    AND  e.timestamp <= '{end_date}'
    {clinic_condition}
    AND p.clinicId IN ('83a53614-f7cc-4ebf-a117-3beceda02747','fff0ed73-282e-4f22-a7d9-e9396aa0af99','62ca07e4-b021-4324-a4ca-35f95831c281', '152aa029-a643-4ee8-b5c6-246294603453')
   
    GROUP BY e.payload.clientId,p.clinicId
""")
display(referred)

referred_list = referred.select("sourceId").rdd.flatMap(lambda x: x).collect()
referred_list = [str(source_id) for source_id in referred_list]

#print(referred_list)

#referred_df = spark.createDataFrame([(source_id,) for source_id in referred_list], ["sourceid"])
#display(referred_df)

sourceId,clinicId,ReferralDate
662bae46-86fc-4b43-bcd2-8d009e8f2966,152aa029-a643-4ee8-b5c6-246294603453,2023-11-17
19d528cc-9354-4875-b754-e9d1b32c5cfb,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-11
446ea9f9-cc0c-4c05-a1e2-96c612955c78,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-25
cd6244bb-ee8c-40f8-8599-57194f42dd18,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-25
96ab5967-74e8-4032-b26a-9c01d76b1c4c,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-19
caf94a40-fd7a-4a50-b97a-86db95d707f5,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-11
b4b9e2e8-6966-48fc-821f-086000c82eff,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-06
d0efe43f-e990-4b8d-ae2d-b49950461d37,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-11
2f20e3c7-2f37-4686-9016-550784502cbe,83a53614-f7cc-4ebf-a117-3beceda02747,2023-09-11
98e33549-769c-4d4f-a74b-e202ec754590,62ca07e4-b021-4324-a4ca-35f95831c281,2023-12-01


In [0]:

#referred_df = spark.createDataFrame([(source_id,) for source_id in referred_list], ["sourceid"])
#display(referred_df)

In [0]:
# d2c are automatically activated
dtc_activated = spark.sql(f"""
    select 
    distinct 
    
    e.payload.clientId,
    min(date(e.timestamp)) as DateActivated
    from prodeu.events e   
    JOIN prodeu.patients p on e.payload.clientId = p.clientId
    WHERE e.type = 'clinician-patient-referred'
    AND p.clinicId in ('62ca07e4-b021-4324-a4ca-35f95831c281','152aa029-a643-4ee8-b5c6-246294603453')
    GROUP BY e.payload.clientId
""")
display(dtc_activated)

clientId,DateActivated
662bae46-86fc-4b43-bcd2-8d009e8f2966,2023-11-17
42e0c7e1-fba3-4978-a7fa-862ad9bc4738,2023-11-27
98e33549-769c-4d4f-a74b-e202ec754590,2023-12-01


In [0]:
# referral patient using clinician-patient-referred events
# maybe add additional patients (from previous referral process)


# activated ONLY FROM referral_list
# and IN e.payload.sourceId {format(tuple(referred_list))}


if referred_list:
    # If not empty, include the IN clause in the SQL query
    if len(referred_list) == 1:
        query_condition = f"e.payload.sourceId = '{referred_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(referred_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"


activated = spark.sql(f"""
    select 
    distinct 
    e.payload.sourceId,
    --e.payload.clientId,
    min(date(e.timestamp)) as DateActivated
    from prodeu.events e   
    WHERE e.type = 'videoask-responses-sent'
    and e.payload.videoaskAnswersInput.title = 'Welcome'
    and e.payload.videoaskAnswersInput.questionAnswers.`Q-Wrap_up_options`.poll_options.content[0] is not null
    AND {query_condition}
    GROUP BY e.payload.sourceId
""")

dtc_activated = spark.sql(f"""
    select 
    distinct 
    e.payload.clientId,
    min(date(e.timestamp)) as DateActivated
    from prodeu.events e   
    JOIN prodeu.patients p on e.payload.clientId = p.clientId
    WHERE e.type = 'clinician-patient-referred'
    AND p.clinicId in ('62ca07e4-b021-4324-a4ca-35f95831c281','152aa029-a643-4ee8-b5c6-246294603453')
    GROUP BY e.payload.clientId
""")
#display(dtc_activated)

activated = activated.union(dtc_activated)

display(activated)

activated_list = activated.select("sourceId").rdd.flatMap(lambda x: x).collect()
activated_list = [str(source_id) for source_id in activated_list]

# add previously referred patients?

print(activated_list)

sourceId,DateActivated
43881376-f4fe-4789-a19a-bdfd550bd916,2023-12-09
2f20e3c7-2f37-4686-9016-550784502cbe,2023-09-21
ea836c41-5a72-4bd8-b11c-43a5516eac5f,2023-09-15
cd6244bb-ee8c-40f8-8599-57194f42dd18,2023-10-30
19d528cc-9354-4875-b754-e9d1b32c5cfb,2023-10-23
446ea9f9-cc0c-4c05-a1e2-96c612955c78,2023-10-31
42e0c7e1-fba3-4978-a7fa-862ad9bc4738,2023-11-27
98e33549-769c-4d4f-a74b-e202ec754590,2023-12-01
662bae46-86fc-4b43-bcd2-8d009e8f2966,2023-11-17


['43881376-f4fe-4789-a19a-bdfd550bd916', '2f20e3c7-2f37-4686-9016-550784502cbe', 'ea836c41-5a72-4bd8-b11c-43a5516eac5f', 'cd6244bb-ee8c-40f8-8599-57194f42dd18', '19d528cc-9354-4875-b754-e9d1b32c5cfb', '446ea9f9-cc0c-4c05-a1e2-96c612955c78', '662bae46-86fc-4b43-bcd2-8d009e8f2966', '42e0c7e1-fba3-4978-a7fa-862ad9bc4738', '98e33549-769c-4d4f-a74b-e202ec754590']


In [0]:
# Add an empty string to the existing activated_list
activated_list_with_no_filter = ["Unselected"] + activated_list

# Create the dropdown filter widget with the modified referred list
dbutils.widgets.dropdown("Activated Patient List", activated_list_with_no_filter[0], activated_list_with_no_filter)

# Retrieve the selected patient name from the widget
selected_activated_patient = dbutils.widgets.get("Activated Patient List")

# Check if an empty string was selected
if selected_activated_patient == "Unselected":
    # Handle the case when no patient is selected (no filtering)
    # For example, you can print a message or set a flag to indicate no filtering.
    no_filtering = True
    filtered_patient = None  # Optionally set a default value
else:
    # Use the selected patient name in your code
    no_filtering = False
    filtered_patient = selected_activated_patient
print(selected_activated_patient)

Unselected


In [0]:
if selected_activated_patient == "Unselected":
    # Set the query condition to an empty string or a condition that always evaluates to true
    activated_query_condition = ""
else:
    # Use the selected patient name in your query condition
    activated_query_condition = f"e.payload.sourceId = '{selected_activated_patient}'"


In [0]:
if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"


if selected_activated_patient == "Unselected":
    # Set the query condition to an empty string or a condition that always evaluates to true
    activated_query_condition = ""
else:
    # Use the selected patient name in your query condition
    activated_query_condition = f" and e.payload.sourceId = '{selected_activated_patient}'"

testing = spark.sql(f"""
   SELECT 
   e.payload.sourceId 
   FROM prodeu.events e
   WHERE e.tenant = 4 
   and {query_condition}
   {activated_query_condition}            
                    
""")
display(testing)

sourceId
2f20e3c7-2f37-4686-9016-550784502cbe
2f20e3c7-2f37-4686-9016-550784502cbe
2f20e3c7-2f37-4686-9016-550784502cbe
2f20e3c7-2f37-4686-9016-550784502cbe
2f20e3c7-2f37-4686-9016-550784502cbe
2f20e3c7-2f37-4686-9016-550784502cbe
2f20e3c7-2f37-4686-9016-550784502cbe
2f20e3c7-2f37-4686-9016-550784502cbe
2f20e3c7-2f37-4686-9016-550784502cbe
2f20e3c7-2f37-4686-9016-550784502cbe


In [0]:

# Convert both lists to sets for easier set operations
referred_set = set(referred_list)
activated_set = set(activated_list)

# Find patients in referred_list but not in activated_list (i.e., inactivated patients)
inactivated_set = referred_set - activated_set

# Convert the result back to a list if needed
inactivated_list = list(inactivated_set)

# Print or use the inactivated_list as needed
print(inactivated_list)


['b4b9e2e8-6966-48fc-821f-086000c82eff', 'e8a7d975-6609-4d09-a2c9-d49318d0bb10', '96ab5967-74e8-4032-b26a-9c01d76b1c4c', 'a2bb3e9f-d1bc-45e8-8a71-05a3463006a5', '90109a4e-ae15-4295-87dd-bddda3251c2b', '5d3de71a-a6eb-44e0-ab02-e005afba3e72', '5fbe3618-27d9-4223-a6b1-1403fe2ed7b8', 'caf94a40-fd7a-4a50-b97a-86db95d707f5', 'd0efe43f-e990-4b8d-ae2d-b49950461d37']


In [0]:
# referred but not activated
# Add an empty string to the existing referred_list
inactivated_list_with_no_filter = ["Unselected"] + inactivated_list

# Create the dropdown filter widget with the modified referred list
dbutils.widgets.dropdown("Inactivated Patient List", inactivated_list_with_no_filter[0], inactivated_list_with_no_filter)

# Retrieve the selected patient name from the widget
selected_inactivated_patient = dbutils.widgets.get("Inactivated Patient List")

# Check if an empty string was selected
if selected_inactivated_patient == "Unselected":
    # Handle the case when no patient is selected (no filtering)
    # For example, you can print a message or set a flag to indicate no filtering.
    no_filtering = True
    filtered_patient = None  # Optionally set a default value
else:
    # Use the selected patient name in your code
    no_filtering = False
    filtered_patient = selected_inactivated_patient
print(selected_inactivated_patient)

Unselected


In [0]:
# referred to activated in days
referred_activated_days = referred.join(activated, on = 'sourceId', how = 'left')

# Assuming 'referred_activated_days' DataFrame has 'ReferralDate' and 'DateActivated' columns
referred_activated_days = referred_activated_days.withColumn('ReferralDate', to_date(col('ReferralDate')))
referred_activated_days = referred_activated_days.withColumn('DateActivated', to_date(col('DateActivated')))

# Calculate the difference in days
referred_activated_days = referred_activated_days.withColumn('ReferralToActivateDays', datediff(col('DateActivated'),col('ReferralDate')))

display(referred_activated_days)


sourceId,clinicId,ReferralDate,DateActivated,ReferralToActivateDays
2f20e3c7-2f37-4686-9016-550784502cbe,83a53614-f7cc-4ebf-a117-3beceda02747,2023-09-11,2023-09-21,10
98e33549-769c-4d4f-a74b-e202ec754590,62ca07e4-b021-4324-a4ca-35f95831c281,2023-12-01,2023-12-01,0
42e0c7e1-fba3-4978-a7fa-862ad9bc4738,62ca07e4-b021-4324-a4ca-35f95831c281,2023-11-27,2023-11-27,0
ea836c41-5a72-4bd8-b11c-43a5516eac5f,83a53614-f7cc-4ebf-a117-3beceda02747,2023-09-11,2023-09-15,4
43881376-f4fe-4789-a19a-bdfd550bd916,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-12-05,2023-12-09,4
662bae46-86fc-4b43-bcd2-8d009e8f2966,152aa029-a643-4ee8-b5c6-246294603453,2023-11-17,2023-11-17,0
19d528cc-9354-4875-b754-e9d1b32c5cfb,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-11,2023-10-23,12
446ea9f9-cc0c-4c05-a1e2-96c612955c78,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-25,2023-10-31,6
cd6244bb-ee8c-40f8-8599-57194f42dd18,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-10-25,2023-10-30,5
a2bb3e9f-d1bc-45e8-8a71-05a3463006a5,fff0ed73-282e-4f22-a7d9-e9396aa0af99,2023-12-05,null,null


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.types import FloatType


# d2c clinics are excluded
excluded_clinic_ids = ['152aa029-a643-4ee8-b5c6-246294603453', '62ca07e4-b021-4324-a4ca-35f95831c281']
filtered_referred_activated_days = referred_activated_days.filter(~col('clinicId').isin(excluded_clinic_ids))

if filtered_referred_activated_days.filter(col('ReferralToActivateDays').isNotNull()).count() > 0:
    # Calculate mean
    mean_value = filtered_referred_activated_days.select(round(mean(col('ReferralToActivateDays')), 2)).collect()
    
    # Calculate min, max, and median
    min_value = filtered_referred_activated_days.select(min(col('ReferralToActivateDays')).alias('min')).collect()
    max_value = filtered_referred_activated_days.select(max(col('ReferralToActivateDays')).alias('max')).collect()
    median_value = filtered_referred_activated_days.approxQuantile('ReferralToActivateDays', [0.5], 0.25)[0]
    
    # Create DataFrames for display
    mean_df = spark.createDataFrame(mean_value, ["MeanReferralToActivateDays"])
    min_df = spark.createDataFrame(min_value, ["MinReferralToActivateDays"])
    max_df = spark.createDataFrame(max_value, ["MaxReferralToActivateDays"])
    median_df = spark.createDataFrame([(median_value,)], ["MedianReferralToActivateDays"])
    
    # Display the DataFrames
    display(mean_df)
    display(min_df)
    display(max_df)
    display(median_df)
else:
    # If there are no non-null values, create and display empty DataFrames
    empty_data = [(None,)]
    empty_schema = [StructField("MeanReferralToActivateDays", FloatType(), True)]
    empty_df = spark.createDataFrame(empty_data, StructType(empty_schema))
    display(empty_df)



MeanReferralToActivateDays
6.83


Databricks visualization. Run in Databricks to view.

MinReferralToActivateDays
4


MaxReferralToActivateDays
12


MedianReferralToActivateDays
4.0


In [0]:
# {query_condition}
# behaviour of activated patient
if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"

# number of tasks completed by week, 
# number of minutes spent on phone (dialpad hangup event)
# number of phone calls?
# number of messages received

activated_behaviour = spark.sql(f"""
    SELECT 
    CONCAT(YEAR(e.timestamp), "-W", LPAD(WEEKOFYEAR(e.timestamp), 2, "0")) as week,
    e.payload.sourceId

    FROM prodeu.events e
    WHERE {query_condition}  
    GROUP BY sourceId, week                           


""")
display(activated_behaviour)

week,sourceId
2023-W40,ea836c41-5a72-4bd8-b11c-43a5516eac5f
2023-W43,19d528cc-9354-4875-b754-e9d1b32c5cfb
2023-W45,ea836c41-5a72-4bd8-b11c-43a5516eac5f
2023-W44,19d528cc-9354-4875-b754-e9d1b32c5cfb
2023-W41,ea836c41-5a72-4bd8-b11c-43a5516eac5f
2023-W45,2f20e3c7-2f37-4686-9016-550784502cbe
2023-W43,446ea9f9-cc0c-4c05-a1e2-96c612955c78
2023-W41,19d528cc-9354-4875-b754-e9d1b32c5cfb
2023-W46,19d528cc-9354-4875-b754-e9d1b32c5cfb
2023-W42,2f20e3c7-2f37-4686-9016-550784502cbe


In [0]:
# Counting number of patients who completed DA
# --t.t_total_count IN {format(tuple(activated_list))}
# {query_condition}
if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"


if referred_list:
    # If not empty, include the IN clause in the SQL query
    if len(referred_list) == 1:
        query_condition_two = f"e.payload.sourceId = '{referred_list[0]}'"
    else:
        query_condition_two = f"e.payload.sourceId IN {format(tuple(referred_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition_two = "1 = 0"



DACompletedNames = spark.sql(f"""
WITH t AS (
    SELECT 
    distinct e.payload.sourceId as t_total_count,
    e.type as type,
    max(date(e.timestamp)) DateCompletedDA,
    COUNT (DISTINCT e.payload.typeformAnswersInput.title) AS TypeformDACompleted
    FROM prodeu.events e
    where e.type = 'typeform-responses-sent'
    AND e.payload.typeformAnswersInput.title IN ('Profile','Values', 'Treatment Preference', 'Home Life')
    GROUP BY e.payload.sourceId,e.type
    HAVING TypeformDACompleted = 4
    ),


v AS (
    select 
    DISTINCT e.payload.sourceId as v_total_count
    from prodeu.events e
    WHERE e.type = 'videoask-responses-sent'
    AND e.payload.videoaskAnswersInput.title = 'Welcome'
    AND {query_condition_two}
    AND e.payload.videoaskAnswersInput.questionAnswers.`Q-Wrap_up_options`.poll_options.content[0] is not null
)
SELECT 
    DISTINCT v.v_total_count as sourceId,
    t.DateCompletedDA as DateCompletedDA
    --CASE WHEN t.type is not null then 'Completed'
    --ELSE 'Not Complete' end as DAStatus
    --DISTINCT t.t_total_count as Completed_DA
FROM v
JOIN t ON t.t_total_count = v.v_total_count

""")
display(DACompletedNames)


# funnel analysis

funnel = referred.join(activated, on = 'sourceId', how = 'left')
funnel = funnel.join(DACompletedNames, on = 'sourceId', how = 'left')
funnel = funnel.withColumn("ReferredStatus", when(col("DateActivated").isNotNull(), "Referred").otherwise(None))
funnel = funnel.withColumn("ActivatedStatus", when(col("DateActivated").isNotNull(), "Activated").otherwise("Not Activated"))
funnel = funnel.withColumn("DACompletedStatus", when(col("DateCompletedDA").isNotNull(), "Completed DA").otherwise(None))

funnel = funnel.fillna({"ReferredStatus": "Referred"})
funnel = funnel.fillna({"DACompletedStatus": "Not Completed DA"})
#display(funnel)




sourceId,DateCompletedDA


In [0]:

# Create a dictionary to map clinicId to clinic names
# Create a dictionary to map clinicId to clinic names
clinic_names = {
    "83a53614-f7cc-4ebf-a117-3beceda02747": {"name": "Clinic B", "color": "#3366CC"},
    "fff0ed73-282e-4f22-a7d9-e9396aa0af99": {"name": "Clinic N", "color": "#FD3216"},
    "62ca07e4-b021-4324-a4ca-35f95831c281": {"name": "Clinic U", "color": "#FF9616"},
    "152aa029-a643-4ee8-b5c6-246294603453": {"name": "Clinic C", "color": "#1CA71C"}
}

funnel_pandas = funnel.toPandas()
fig = go.Figure()

# Group the data by 'clinicId'
grouped_data = funnel_pandas.groupby('clinicId')

# Loop through each clinicId group
for clinic_id, data in grouped_data:
    clinic_info = clinic_names.get(clinic_id, {"name": "Unknown Clinic", "color": "gray"})

    # Count the occurrences of each stage within the group
    referred_count = data[data['ReferredStatus'] == 'Referred']['ReferredStatus'].count()
    activated_count = data[data['ActivatedStatus'] == 'Activated']['ActivatedStatus'].count()

    # Create a funnel trace for the clinic with the clinic's name as the name
    fig.add_trace(go.Funnel(
        name=str(clinic_info["name"]),  # Convert the name to a string
        y=["ReferredStatus", "ActivatedStatus"],
        x=[referred_count, activated_count],
        textinfo="value+percent initial",
        marker=dict(color=clinic_info["color"])  # Set the color for the trace
    ))

fig.update_layout(
    xaxis_title="Counts",
    funnelmode="stack",
)
# Show the funnel chart
fig.show()


In [0]:
# Avg Days to Activate

# Calculate the mean of the 'ReferralToActivateDays' column and round it to 2 decimals
#mean_referral_to_activate_days = referred_activated_days.select(round(mean('ReferralToActivateDays'), 2)).collect()[0][0]
#mean_df = spark.createDataFrame([(mean_referral_to_activate_days,)], ["MeanReferralToActivateDays"])
#display(mean_df)

#print(mean_referral_to_activate_days)



In [0]:
# Check if the activated_list is empty
if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"

# Use the query_condition in your SQL query
MAU_Activated = spark.sql(f"""
   SELECT 
   date_format(e.timestamp, 'yyyy-MM') as Month,
   count(distinct e.payload.sourceId) as MAU
   FROM prodeu.events e                       
   WHERE {query_condition}
   AND e.timestamp >= '{start_date}' 
   AND e.timestamp <= '{end_date}'
   GROUP BY Month
""")
display(MAU_Activated)


Month,MAU
2023-12,9
2023-11,7
2023-10,5
2023-09,2


Databricks visualization. Run in Databricks to view.

In [0]:
# Check if the activated_list is empty
if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"


# DAU of Activated users
DAU_Activated = spark.sql(f"""
   SELECT 
   date(e.timestamp) as Date,
   count (distinct e.payload.sourceId) as DAU
   FROM prodeu.events e                       
   WHERE {query_condition}
   AND e.timestamp >= '{start_date}' 
   AND  e.timestamp <= '{end_date}'
   GROUP BY Date
""")
display(DAU_Activated)

Date,DAU
2023-09-14,2
2023-11-08,3
2023-11-22,5
2023-09-19,1
2023-11-29,4
2023-12-10,1
2023-09-27,2
2023-11-17,4
2023-11-18,1
2023-10-12,1


Databricks visualization. Run in Databricks to view.

In [0]:
# patient mapping
# might need to manually input?
# including initials?

# 'sourceid':'initials'
#patient_mappting = ['':'',]

# map patient name
#map_patient_name_udf = udf(lambda sourceId: patient_mapping.get(sourceId, 'Unknown'))

In [0]:
# DAU of activated patients
# cumulative total of activated patients
# inactive patients

In [0]:
# welcome completed and by date

if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"

date_activated = spark.sql(f"""
    select
    distinct e.payload.sourceid,
    first(date(e.timestamp)) as date
    --e.payload.welcomeVideoaskEventInput 
    from prodeu.events e 
    WHERE e.type = 'videoask-responses-sent'
    AND e.payload.videoaskAnswersInput.title = 'Welcome'  
    AND {query_condition}  
    GROUP BY e.payload.sourceid
                
                        
""")
display(date_activated)

sourceid,date
43881376-f4fe-4789-a19a-bdfd550bd916,2023-12-09
2f20e3c7-2f37-4686-9016-550784502cbe,2023-09-21
ea836c41-5a72-4bd8-b11c-43a5516eac5f,2023-09-15
cd6244bb-ee8c-40f8-8599-57194f42dd18,2023-10-30
19d528cc-9354-4875-b754-e9d1b32c5cfb,2023-10-23
446ea9f9-cc0c-4c05-a1e2-96c612955c78,2023-10-31


In [0]:
# meaingfully engaged
if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"


SMS_Meaningfully_engaged = spark.sql(f"""
    SELECT 
        smsSend.sourceId,
        SUM(smsSend.smsSend) AS totalSMSsent,
        SUM(smsClick.smsClick) AS totalSMSclick,
        CASE 
            WHEN (SUM(smsClick.smsClick) / SUM(smsSend.smsSend) * 100) > 100 
            THEN 100 
            ELSE ROUND((SUM(smsClick.smsClick) / SUM(smsSend.smsSend) * 100), 2)
        END AS SMS_view_rate
    FROM
        (SELECT 
            e.payload.sourceId,
            COUNT(*) AS smsSend,
            e.payload.iterableEvent.DataFields.workflowId AS workflowId
        FROM PRODEU.events e
        WHERE e.type = 'iterable-smsSend'
            AND e.payload.iterableEvent.DataFields.workflowId IS NOT NULL
            AND {query_condition}
        GROUP BY e.payload.sourceId, e.payload.iterableEvent.DataFields.workflowId) AS smsSend
    LEFT JOIN
        (SELECT 
            e.payload.sourceId,
            COUNT(*) AS smsClick,
            e.payload.iterableEvent.DataFields.workflowId AS workflowId
        FROM PRODEU.events e
        WHERE e.type = 'iterable-smsClick'
            AND e.payload.iterableEvent.DataFields.workflowId IS NOT NULL
            AND {query_condition}
        GROUP BY e.payload.sourceId, e.payload.iterableEvent.DataFields.workflowId) AS smsClick
    ON smsSend.sourceId = smsClick.sourceId
    AND smsSend.workflowId = smsClick.workflowId

    GROUP BY smsSend.sourceId
""")
#display(SMS_Meaningfully_engaged)

# email sent vs email open

Email_Meaningfully_engaged = spark.sql(f"""
    SELECT 
        emailSent.sourceId,
        SUM(emailSent.emailSent) AS totalEmailSent,
        SUM(emailOpen.emailOpen) AS totalEmailOpen,
        CASE 
            WHEN (SUM(emailOpen.emailOpen) / SUM(emailSent.emailSent) * 100) > 100 
            THEN 100 
            ELSE ROUND((SUM(emailOpen.emailOpen) / SUM(emailSent.emailSent) * 100), 2) 
        END AS Email_view_rate
    FROM
        (SELECT 
            e.payload.sourceId,
            COUNT(*) AS emailSent,
            e.payload.iterableEvent.DataFields.workflowId AS workflowId
        FROM PRODEU.events e
        WHERE e.type = 'iterable-emailSend'
            AND e.payload.iterableEvent.DataFields.workflowId IS NOT NULL
            AND {query_condition}
        GROUP BY e.payload.sourceId, e.payload.iterableEvent.DataFields.workflowId) AS emailSent
    LEFT JOIN
        (SELECT 
            e.payload.sourceId,
            COUNT(*) AS emailOpen,
            e.payload.iterableEvent.DataFields.workflowId AS workflowId
        FROM PRODEU.events e
        WHERE e.type = 'iterable-emailOpen'
            AND e.payload.iterableEvent.DataFields.workflowId IS NOT NULL
            AND {query_condition}
        GROUP BY e.payload.sourceId, e.payload.iterableEvent.DataFields.workflowId) AS emailOpen
    ON emailSent.sourceId = emailOpen.sourceId
    AND emailSent.workflowId = emailOpen.workflowId
    GROUP BY emailSent.sourceId
""")
#display(Email_Meaningfully_engaged)

meaningfully_engaged = Email_Meaningfully_engaged.join(SMS_Meaningfully_engaged, on='sourceId', how='full')
meaningfully_engaged = meaningfully_engaged.filter((col('email_view_rate') >= '75') | (col('SMS_view_rate') >= '75'))


filtered_meaningfully_engaged = meaningfully_engaged.filter(col('sourceId').isin(activated_list))

if len(activated_list) > 0:
    rate = filtered_meaningfully_engaged.select('sourceId').distinct().count() / len(activated_list) * 100
else:
    rate = 0.0

rate_df = spark.createDataFrame([Row(rate=rate)])
rate_df = rate_df.withColumn("MeaningfullyEngaged", concat(round(rate_df["rate"], 2).cast("string"), lit("%")))
rate_df = rate_df.drop("rate")
display(rate_df)

MeaningfullyEngaged
33.33%


Databricks visualization. Run in Databricks to view.

In [0]:
# new meanmingfully engaged definition
# Completing one task or send one message every two week, for the last 6 weeks, or since activation = meaningfully engaged 
# Activated only 

if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"





In [0]:
%sql
select 
e.payload.sourceId,
e.payload.clientId,
e.payload.typeformEventInput.definition.title,
e.timestamp
from prodeu.events e
where e.type = 'typeform-completed'
AND e.payload.typeformEventInput.definition.title LIKE '%Call%';

sourceId,clientId,title,timestamp
614bb0d2-9ac1-4dc8-a013-cb34c19e620d,614bb0d2-9ac1-4dc8-a013-cb34c19e620d,Call 1,2023-06-28T21:13:53.069Z
d439f06f-7cbe-4670-a3a9-561855a5eaa8,d439f06f-7cbe-4670-a3a9-561855a5eaa8,Call Log,2023-09-01T22:28:01.607Z
58d32e3a-f2b0-4527-a58c-b9c1e6fc11d4,58d32e3a-f2b0-4527-a58c-b9c1e6fc11d4,Call Log,2023-09-01T22:28:30.664Z
c5a3b526-acfb-4543-ace6-171f4e999ba1,c5a3b526-acfb-4543-ace6-171f4e999ba1,Call Log,2023-09-01T22:32:51.894Z
64e8d75b-57c5-446a-abc2-cc59281fbedd,64e8d75b-57c5-446a-abc2-cc59281fbedd,Call Log,2023-09-01T22:37:50.315Z
475f88e4-9f1d-4c8c-b141-e297dd37024c,475f88e4-9f1d-4c8c-b141-e297dd37024c,Call Log,2023-09-13T17:06:58.575Z
17260bb0-f944-4eb6-bf01-1efae69b80cf,17260bb0-f944-4eb6-bf01-1efae69b80cf,Call Log,2023-09-13T17:08:05.156Z
475f88e4-9f1d-4c8c-b141-e297dd37024c,475f88e4-9f1d-4c8c-b141-e297dd37024c,Call Log,2023-09-13T21:36:55.570Z
17260bb0-f944-4eb6-bf01-1efae69b80cf,17260bb0-f944-4eb6-bf01-1efae69b80cf,Call Log,2023-09-13T23:46:19.654Z
cfa87a02-a85f-474d-baf1-7ed5969884cb,cfa87a02-a85f-474d-baf1-7ed5969884cb,Call Log,2023-09-14T21:05:31.139Z


In [0]:
# completed at least one task every two weeks
# completed or message sent

meaningfully_egngaged_task = spark.sql(f"""
   SELECT 
   date(e.timestamp) as Date,
   e.payload.clientid,
   COUNT(e.payload.clientid) filter (WHERE e.type = 'program-task-completed') as completed,
   COUNT(e.payload.clientid) filter (WHERE e.type = 'program-task-sent') as sent
   FROM prodeu.events e
   WHERE {query_condition}
   GROUP BY  Date, clientid                                 
                                       
""")
display(meaningfully_egngaged_task)

Date,clientid,completed,sent
2023-11-22,2f20e3c7-2f37-4686-9016-550784502cbe,0,1
2023-11-22,662bae46-86fc-4b43-bcd2-8d009e8f2966,0,0
2023-12-11,42e0c7e1-fba3-4978-a7fa-862ad9bc4738,0,1
2023-12-01,98e33549-769c-4d4f-a74b-e202ec754590,1,0
2023-09-16,ea836c41-5a72-4bd8-b11c-43a5516eac5f,0,0
2023-12-06,cd6244bb-ee8c-40f8-8599-57194f42dd18,0,1
2023-12-12,43881376-f4fe-4789-a19a-bdfd550bd916,0,1
2023-11-24,2f20e3c7-2f37-4686-9016-550784502cbe,0,0
2023-11-27,42e0c7e1-fba3-4978-a7fa-862ad9bc4738,1,0
2023-12-04,ea836c41-5a72-4bd8-b11c-43a5516eac5f,0,1


In [0]:
num_appointments = spark.sql(f"""
   SELECT
   
   --e.payload.clientId as PatientId,
   e.payload.sourceId,
   date(e.payload.appointmentDetails.datetime) as AppointmentDate
   FROM prodeu.events e
   WHERE e.type = 'appointment-scheduled'
   --AND {query_condition}
   AND e.payload.appointmentDetails.datetime is not null
   --and e.payload.sourceId = 'ea836c41-5a72-4bd8-b11c-43a5516eac5f'
   --AND e.payload.appointmentDetails.datetime >= current_date()                     
                             
""")
display(num_appointments)

sourceId,AppointmentDate
24b668d7-f458-43ce-af83-5f98418f62a8,2023-06-12
58756224-f230-4c66-b5d4-ca455b70a722,2023-06-28
58756224-f230-4c66-b5d4-ca455b70a722,2023-08-23
23fe1a80-2583-4ca0-93f0-511e679114c0,2023-08-23
23fe1a80-2583-4ca0-93f0-511e679114c0,2023-08-30
58756224-f230-4c66-b5d4-ca455b70a722,2023-07-14
58756224-f230-4c66-b5d4-ca455b70a722,2023-07-25
58756224-f230-4c66-b5d4-ca455b70a722,2023-08-15
58756224-f230-4c66-b5d4-ca455b70a722,2023-08-15
24b668d7-f458-43ce-af83-5f98418f62a8,2023-11-07


In [0]:
# number of patients with upcoming appointments?
# change the filter to current date
if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"


num_appointments = spark.sql(f"""
   SELECT
   
   e.payload.clientId as PatientId,
   date(e.payload.appointmentDetails.datetime) as AppointmentDate
   FROM prodeu.events e
   WHERE e.type = 'appointment-scheduled'
   AND {query_condition}
   
   AND e.payload.appointmentDetails.datetime is not null
   --AND e.payload.appointmentDetails.datetime >= current_date()                     
                             
""")
display(num_appointments)

PatientId,AppointmentDate


Databricks visualization. Run in Databricks to view.

In [0]:
print(inactivated_list)

['b4b9e2e8-6966-48fc-821f-086000c82eff', 'e8a7d975-6609-4d09-a2c9-d49318d0bb10', '96ab5967-74e8-4032-b26a-9c01d76b1c4c', 'a2bb3e9f-d1bc-45e8-8a71-05a3463006a5', '90109a4e-ae15-4295-87dd-bddda3251c2b', '5d3de71a-a6eb-44e0-ab02-e005afba3e72', '5fbe3618-27d9-4223-a6b1-1403fe2ed7b8', 'caf94a40-fd7a-4a50-b97a-86db95d707f5', 'd0efe43f-e990-4b8d-ae2d-b49950461d37']


In [0]:
# Use a condition that always evaluates to true when activated_list is empty
if activated_list:
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{inactivated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(inactivated_list))}"
else:
    query_condition = "1 = 1"  # This condition is always true

# Check if inactivated_list is not empty
if inactivated_list:
    Inactivated_patient_metrics = spark.sql(f"""
       SELECT
       DISTINCT e.payload.sourceId as InactivatedPatient,
       CONCAT(
           ROUND(
               (count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailBounce') /
               count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailSend')) * 100,
               2
           ),
           '%'
       )  as EmailBounceRate,
       CONCAT(
           ROUND(
               (1 - (
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailOpen') /
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailSend')
               )) * 100,
               2
           ),
           '%'
       )  as NoEmailOpenRate,

       CONCAT(
           ROUND(
               (1 - (
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailClick') /
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailOpen')
               )) * 100,
               2
           ),
           '%'
       )  as NoEmailClickRate,

       CONCAT(
           ROUND(
               (1 - (
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-smsClick') /
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-smsSend')
               )) * 100,
               2
           ),
           '%'
       )  as NoSMSClickRate,

       count( e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailSend') NumEmailSent,
       count( e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-smsSend') as NumSMSSent
       FROM prodeu.events e
       WHERE {query_condition} 
       GROUP BY e.payload.sourceId
    """)
    display(Inactivated_patient_metrics)
else:
    print("No inactivated patients to query.")


InactivatedPatient,EmailBounceRate,NoEmailOpenRate,NoEmailClickRate,NoSMSClickRate,NumEmailSent,NumSMSSent
e8a7d975-6609-4d09-a2c9-d49318d0bb10,100.0%,100.0%,null,100.0%,5,8
5fbe3618-27d9-4223-a6b1-1403fe2ed7b8,100.0%,100.0%,null,100.0%,4,4
a2bb3e9f-d1bc-45e8-8a71-05a3463006a5,100.0%,100.0%,null,100.0%,4,4
90109a4e-ae15-4295-87dd-bddda3251c2b,0.0%,0.0%,0.0%,100.0%,8,6
5d3de71a-a6eb-44e0-ab02-e005afba3e72,0.0%,100.0%,null,75.0%,12,6
b4b9e2e8-6966-48fc-821f-086000c82eff,0.0%,25.0%,66.67%,100.0%,20,13
d0efe43f-e990-4b8d-ae2d-b49950461d37,0.0%,25.0%,66.67%,100.0%,23,7
96ab5967-74e8-4032-b26a-9c01d76b1c4c,0.0%,0.0%,100.0%,100.0%,18,1
caf94a40-fd7a-4a50-b97a-86db95d707f5,0.0%,100.0%,null,100.0%,45,15


In [0]:
# Use a condition that always evaluates to true when activated_list is empty
if activated_list:
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{inactivated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(inactivated_list))}"
else:
    query_condition = "1 = 1"  # This condition is always true

# Check if inactivated_list is not empty
if inactivated_list:
    Inactivated_patient_metrics = spark.sql(f"""
       SELECT
       DISTINCT e.payload.sourceId as InactivatedPatient,
       any_value(CASE 
        WHEN p.clinicId = "83a53614-f7cc-4ebf-a117-3beceda02747" THEN "Clinic B"
        WHEN p.clinicId = "fff0ed73-282e-4f22-a7d9-e9396aa0af99" THEN "Clinic N"
        END) as Clinic,
       CONCAT(
           ROUND(
               (count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailBounce') /
               count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailSend')) * 100,
               2
           ),
           '%'
       )  as EmailBounceRate,
       CONCAT(
           ROUND(
               (1 - (
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailOpen') /
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailSend')
               )) * 100,
               2
           ),
           '%'
       )  as NoEmailOpenRate,

       CONCAT(
           ROUND(
               (1 - (
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailClick') /
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailOpen')
               )) * 100,
               2
           ),
           '%'
       )  as NoEmailClickRate,

       CONCAT(
           ROUND(
               (1 - (
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-smsClick') /
                   count(distinct e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-smsSend')
               )) * 100,
               2
           ),
           '%'
       )  as NoSMSClickRate,

       count( e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-emailSend') NumEmailSent,
       count( e.payload.iterableEvent.datafields.messageId) filter(where e.type = 'iterable-smsSend') as NumSMSSent
       FROM prodeu.events e
       LEFT JOIN prodeu.patients p on e.payload.sourceId = p.clientId
       WHERE {query_condition} 
       GROUP BY e.payload.sourceId 
    """)
    display(Inactivated_patient_metrics)
else:
    print("No inactivated patients to query.")


InactivatedPatient,Clinic,EmailBounceRate,NoEmailOpenRate,NoEmailClickRate,NoSMSClickRate,NumEmailSent,NumSMSSent
5d3de71a-a6eb-44e0-ab02-e005afba3e72,Clinic N,0.0%,100.0%,null,75.0%,12,6
5fbe3618-27d9-4223-a6b1-1403fe2ed7b8,Clinic N,100.0%,100.0%,null,100.0%,4,4
90109a4e-ae15-4295-87dd-bddda3251c2b,Clinic N,0.0%,0.0%,0.0%,100.0%,8,6
96ab5967-74e8-4032-b26a-9c01d76b1c4c,Clinic N,0.0%,0.0%,100.0%,100.0%,18,1
a2bb3e9f-d1bc-45e8-8a71-05a3463006a5,Clinic N,100.0%,100.0%,null,100.0%,4,4
b4b9e2e8-6966-48fc-821f-086000c82eff,Clinic N,0.0%,25.0%,66.67%,100.0%,20,13
caf94a40-fd7a-4a50-b97a-86db95d707f5,Clinic N,0.0%,100.0%,null,100.0%,45,15
d0efe43f-e990-4b8d-ae2d-b49950461d37,Clinic N,0.0%,25.0%,66.67%,100.0%,23,7
e8a7d975-6609-4d09-a2c9-d49318d0bb10,Clinic N,100.0%,100.0%,null,100.0%,5,8


In [0]:
# Inactivated individual 

In [0]:
# Q-Interest_connecting_others
# Q-Top_3_values
# Q-Main_transportation
# Q-Decision_support

In [0]:
print(activated_list)

['43881376-f4fe-4789-a19a-bdfd550bd916', '2f20e3c7-2f37-4686-9016-550784502cbe', 'ea836c41-5a72-4bd8-b11c-43a5516eac5f', 'cd6244bb-ee8c-40f8-8599-57194f42dd18', '19d528cc-9354-4875-b754-e9d1b32c5cfb', '446ea9f9-cc0c-4c05-a1e2-96c612955c78', '662bae46-86fc-4b43-bcd2-8d009e8f2966', '42e0c7e1-fba3-4978-a7fa-862ad9bc4738', '98e33549-769c-4d4f-a74b-e202ec754590']


In [0]:
# d2c are automatically activated
dtc_activated = spark.sql(f"""
    select 
    distinct 
    e.payload.clientId,
    min(date(e.timestamp)) as DateActivated
    from prodeu.events e   
    JOIN prodeu.patients p on e.payload.clientId = p.clientId
    WHERE e.type = 'clinician-patient-referred'
    AND p.clinicId in ('62ca07e4-b021-4324-a4ca-35f95831c281','152aa029-a643-4ee8-b5c6-246294603453')
    GROUP BY e.payload.clientId
""")
display(dtc_activated)

clientId,DateActivated
662bae46-86fc-4b43-bcd2-8d009e8f2966,2023-11-17
42e0c7e1-fba3-4978-a7fa-862ad9bc4738,2023-11-27
98e33549-769c-4d4f-a74b-e202ec754590,2023-12-01


In [0]:
videoask_onboarding = spark.sql(f"""
   SELECT 
   distinct
   e.payload.sourceId,
   CASE 
   WHEN p.clinicId = "83a53614-f7cc-4ebf-a117-3beceda02747" THEN "Clinic B"
   WHEN p.clinicId = "fff0ed73-282e-4f22-a7d9-e9396aa0af99" THEN "Clinic N"
   WHEN p.clinicId = '62ca07e4-b021-4324-a4ca-35f95831c281' THEN 'Clinic U'
   WHEN p.clinicId = '152aa029-a643-4ee8-b5c6-246294603453' THEN 'Clinic C'
   END as Clinic,
   CASE 
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-location`.poll_options.content IS NOT NULL 
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-location`.poll_options.content[0]
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-location`.poll_options.content IS NOT NULL 
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-location`.poll_options.content[0]
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-nurseMD`.poll_options.content IS NOT NULL
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-nurseMD`.poll_options.content[0]
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-1on1_calls`.poll_options.content IS NOT NULL
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-1on1_calls`.poll_options.content[0]
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-how_do_you_work_together`.poll_options.content IS NOT NULL
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-how_do_you_work_together`.poll_options.content [0]

   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-nurseMD`.poll_options.content IS NOT NULL
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-nurseMD`.poll_options.content[0]
   END AS InitialLocation,
   

   CASE 
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Treatment_preference_dialysis_type`.poll_options[0].content IS NOT NULL 
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Treatment_preference_dialysis_type`.poll_options[0].content
   END AS InitialDialysisType,


    CAST(e.payload.videoaskAnswersInput.questionAnswers.`Q-Preference_confidence`.input_text AS INT) AS InitialConfidence,
    CAST(
    CASE
        WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-guide_help`.input_text IS NOT NULL
        THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-guide_help`.input_text 
        WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-why_guide`.input_text IS NOT NULL
        THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-why_guide`.input_text 
        WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-how_LifePlan_works`.input_text IS NOT NULL
        THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-how_LifePlan_works`.input_text
        WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-is_LifePlan_for_me`.input_text IS NOT NULL
        THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-is_LifePlan_for_me`.input_text
    END AS INT) AS InitialCKDUnderstanding,

   --e.payload.videoaskAnswersInput.questionAnswers,
   e.payload.videoaskAnswersInput.questionAnswers.`Q-Open_to_transplant`.poll_option_content AS InitialTransplant,
   e.payload.videoaskAnswersInput.questionAnswers.`Q-Program_days`.poll_option_content AS ProgramDays,
   e.payload.videoaskAnswersInput.questionAnswers.`Q-Comms_pref`.poll_option_content AS Comms_Pref,
   e.payload.videoaskAnswersInput.questionAnswers.`Q-Wrap_up_options`.poll_option_content AS Call

  -- e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-Q-Initial_treatment_location_preference-how_do_you_work_together`,
  -- e.payload.videoaskAnswersInput.questionAnswers.`Q-Challenges_anxieties-poor_understanding`
   FROM prodeu.events e
   LEFT JOIN prodeu.patients p on e.payload.sourceId = p.clientId
   WHERE e.type = 'videoask-responses-sent'  
   AND e.payload.videoaskAnswersInput.title IN ( 'Welcome','Welcome - D2C')
   --AND {query_condition}        
                                      
""")

display(videoask_onboarding)

sourceId,Clinic,InitialLocation,InitialDialysisType,InitialConfidence,InitialCKDUnderstanding,InitialTransplant,ProgramDays,Comms_Pref,Call
64e8d75b-57c5-446a-abc2-cc59281fbedd,Clinic B,Conservative care,null,3,3,No,Thursday,Email,Get started without a call
42e0c7e1-fba3-4978-a7fa-862ad9bc4738,Clinic U,Home dialysis,Needle access in my arm (Hemodialysis),6,7,Yes,Monday,Email,Get started without a call
98e33549-769c-4d4f-a74b-e202ec754590,Clinic U,Home dialysis,Needle access in my arm (Hemodialysis),0,1,No,Monday,Email,Get started without a call
42e0b26f-a4c4-4761-b4e7-37db17ec5020,Clinic B,Home dialysis,null,2,3,Yes,Tuesday,SMS/Text,Book a call
43881376-f4fe-4789-a19a-bdfd550bd916,Clinic N,In-center dialysis,Tube in my belly (Peritoneal Dialysis),10,6,No,Tuesday,SMS/Text,Get started without a call
ea836c41-5a72-4bd8-b11c-43a5516eac5f,Clinic B,In-center dialysis,Needle access in my arm (Hemodialysis),10,4,Yes,Monday,SMS/Text,Book a call
2f20e3c7-2f37-4686-9016-550784502cbe,Clinic B,Home dialysis,Needle access in my arm (Hemodialysis),5,5,No,Thursday,Email,Get started without a call
e5347001-78b1-4a51-bd39-86ab77182a94,Clinic B,null,null,8,3,Yes,Monday,SMS/Text,Get started without a call
13d23c86-7b14-4e49-8393-bed9de6025be,null,Home dialysis,Tube in my belly (Peritoneal Dialysis),5,5,No,Tuesday,Email,Get started without a call
5efc2431-fb26-47ec-995b-d4224c6b5d16,null,Home dialysis,Tube in my belly (Peritoneal Dialysis),7,8,Yes,Monday,Email,Get started without a call


In [0]:
# videoask responses sent
# detailed view of activated patients
# patient identifier, program progress, DA progress, initial pref, pref confidence, CKD understanding, modality choice, transplant, last lesson,  Program track
# one row = one patient
# days in the program

if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"

videoask_onboarding = spark.sql(f"""
   SELECT 
   distinct
   e.payload.sourceId,
   CASE 
   WHEN p.clinicId = "83a53614-f7cc-4ebf-a117-3beceda02747" THEN "Clinic B"
   WHEN p.clinicId = "fff0ed73-282e-4f22-a7d9-e9396aa0af99" THEN "Clinic N"
   WHEN p.clinicId = '62ca07e4-b021-4324-a4ca-35f95831c281' THEN 'Clinic U'
   WHEN p.clinicId = '152aa029-a643-4ee8-b5c6-246294603453' THEN 'Clinic C'
   END as Clinic,
   CASE 
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-location`.poll_options.content IS NOT NULL 
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-location`.poll_options.content[0]
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-location`.poll_options.content IS NOT NULL 
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-location`.poll_options.content[0]
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-nurseMD`.poll_options.content IS NOT NULL
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-nurseMD`.poll_options.content[0]
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-1on1_calls`.poll_options.content IS NOT NULL
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-1on1_calls`.poll_options.content[0]
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-how_do_you_work_together`.poll_options.content IS NOT NULL
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-how_do_you_work_together`.poll_options.content [0]

   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-nurseMD`.poll_options.content IS NOT NULL
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_location_preference-nurseMD`.poll_options.content[0]
   END AS InitialLocation,
   

   CASE 
   WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Treatment_preference_dialysis_type`.poll_options[0].content IS NOT NULL 
   THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Treatment_preference_dialysis_type`.poll_options[0].content
   END AS InitialDialysisType,


    CAST(e.payload.videoaskAnswersInput.questionAnswers.`Q-Preference_confidence`.input_text AS INT) AS InitialConfidence,
    CAST(
    CASE
        WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-guide_help`.input_text IS NOT NULL
        THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-guide_help`.input_text 
        WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-why_guide`.input_text IS NOT NULL
        THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-why_guide`.input_text 
        WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-how_LifePlan_works`.input_text IS NOT NULL
        THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-how_LifePlan_works`.input_text
        WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-is_LifePlan_for_me`.input_text IS NOT NULL
        THEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Rate_understanding-is_LifePlan_for_me`.input_text
    END AS INT) AS InitialCKDUnderstanding,

   --e.payload.videoaskAnswersInput.questionAnswers,
   e.payload.videoaskAnswersInput.questionAnswers.`Q-Open_to_transplant`.poll_option_content AS InitialTransplant,
   e.payload.videoaskAnswersInput.questionAnswers.`Q-Program_days`.poll_option_content AS ProgramDays,
   e.payload.videoaskAnswersInput.questionAnswers.`Q-Comms_pref`.poll_option_content AS Comms_Pref,
   e.payload.videoaskAnswersInput.questionAnswers.`Q-Wrap_up_options`.poll_option_content AS Call

  -- e.payload.videoaskAnswersInput.questionAnswers.`Q-Initial_treatment_preference-Q-Initial_treatment_location_preference-how_do_you_work_together`,
  -- e.payload.videoaskAnswersInput.questionAnswers.`Q-Challenges_anxieties-poor_understanding`
   FROM prodeu.events e
   LEFT JOIN prodeu.patients p on e.payload.sourceId = p.clientId
   WHERE e.type = 'videoask-responses-sent'  
   AND e.payload.videoaskAnswersInput.title IN ( 'Welcome','Welcome - D2C')
   AND {query_condition}                
                                      
""")

# program progress

typeform_program = spark.sql(f"""
    SELECT 
        DISTINCT
        sourceid,
        TypeformCompletedJourneys
    FROM (
        SELECT 
            DISTINCT e.payload.sourceid AS sourceid,
            SUM(

                CASE 
                    WHEN e.payload.typeformAnswersInput.questionAnswers.`Q-Interest_connecting_others` IS NOT NULL  THEN 1
                    ELSE 0
                END +
                CASE 
                    WHEN e.payload.typeformAnswersInput.questionAnswers.`Q-Top_3_values` IS NOT NULL  THEN 1
                    ELSE 0
                END +
                CASE 
                    WHEN e.payload.typeformAnswersInput.questionAnswers.`Q-Main_transportation` IS NOT NULL  THEN 1
                    ELSE 0
                END 
                
                -- +CASE 
                  --  WHEN e.payload.typeformAnswersInput.questionAnswers.`Q-Decision_support` IS NOT NULL  THEN 1
                  --  ELSE 0
                --END 

            ) AS TypeformCompletedJourneys
        FROM prodeu.events e
        WHERE e.type = 'typeform-responses-sent'
        --AND e.payload.typeformAnswersInput.title IN ('Profile','Values', 'Treatment Preference', 'Home Life')
        AND {query_condition}
        GROUP BY e.payload.sourceid
    )
""")

#display(typeform_program)

videoask_program = spark.sql(f"""
    SELECT 
        sourceid,
        SUM(VideoaskCompletedJourneys) AS VideoaskCompletedJourneys
    FROM (
        SELECT 
            e.payload.sourceid AS sourceid,
            (
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`Q-Wrap_up_options` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Kidneys` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Causes` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Gfr` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Lifestyle_considerations` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`K-3_treatment_choices` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Living_donor` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Dialysis` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-2_pd_methods` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Vascular_access` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) +
                MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Incenter_hd` IS NOT NULL THEN 1
                        ELSE 0
                    END
                )
                +MAX(
                    CASE 
                        WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Conservative_care` IS NOT NULL THEN 1
                        ELSE 0
                    END
                ) 
                --+
                --MAX(
                    --CASE 
                        --WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Cost_or_care` IS NOT NULL THEN 1
                        --ELSE 0
                    --END
                --)
               --  +
               -- MAX(
                   -- CASE 
                     --   WHEN e.payload.videoaskAnswersInput.questionAnswers.`V-Options` IS NOT NULL THEN 1
                      --  ELSE 0
                  --  END
              --  ) 
            ) AS VideoaskCompletedJourneys
        FROM prodeu.events e
        WHERE e.type = 'videoask-responses-sent'
        AND {query_condition}
        GROUP BY e.payload.sourceid
    )
    GROUP BY sourceid
""")


#display(videoask_program)

program_progress = typeform_program.join(videoask_program, on='sourceid', how='fullouter')
program_progress = program_progress.fillna(0)
program_progress = program_progress.withColumn('TotalCompletedJourneys', col('TypeformCompletedJourneys') + col('VideoaskCompletedJourneys'))
program_progress = program_progress.withColumn('ProgramProgress', round((col('TotalCompletedJourneys') / 17) * 100, 2))

#display(program_progress)

# ---
# DA progress videoask + typeform

DAVideoAskProgress = spark.sql(f"""
    select 
    distinct 
    e.payload.sourceId, 
    1 as VideoAskProgress
    from prodeu.events e   
    WHERE e.type = 'videoask-responses-sent'
    AND e.payload.videoaskAnswersInput.title in ('Welcome', 'Welcome - D2C')

    AND e.payload.videoaskAnswersInput.questionAnswers.`Q-Wrap_up_options`.poll_options.content[0] is not null 
    AND {query_condition_two}
    GROUP BY e.payload.sourceId
""")


#display(DAVideoAskProgress)

DATypeformProgress = spark.sql(f"""
    SELECT 
        DISTINCT
        sourceid,
        TypeformDACompleted as TypeformDAProgress
    FROM (
        SELECT 
            DISTINCT e.payload.sourceid AS sourceid,
            SUM(

                CASE 
                    WHEN e.payload.typeformAnswersInput.questionAnswers.`Q-Interest_connecting_others` IS NOT NULL  THEN 1
                    ELSE 0
                END +
                CASE 
                    WHEN e.payload.typeformAnswersInput.questionAnswers.`Q-Top_3_values` IS NOT NULL  THEN 1
                    ELSE 0
                END +
                CASE 
                    WHEN e.payload.typeformAnswersInput.questionAnswers.`Q-Main_transportation` IS NOT NULL  THEN 1
                    ELSE 0
                END 
                
                --+ CASE 
                  --  WHEN e.payload.typeformAnswersInput.questionAnswers.`Q-Decision_support` IS NOT NULL  THEN 1
                    --ELSE 0
                --END 

            ) AS TypeformDACompleted
        FROM prodeu.events e
        WHERE e.type = 'typeform-responses-sent'
        AND e.payload.typeformAnswersInput.title IN ('Profile','Values', 'Treatment Preference', 'Home Life')
        AND {query_condition}
        GROUP BY e.payload.sourceid
    )
""")


#display(DATypeformProgress)

DAProgress = DAVideoAskProgress.join(DATypeformProgress, on = 'sourceId', how = 'outer')
DAProgress = DAProgress.fillna(0)

DAProgress = DAProgress.withColumn(
    'DAProgress',
    concat(
        ((col('VideoAskProgress') + col('TypeformDAProgress')) / 5 * 100).cast("double"),
        lit('%')
    )
)
DAProgress = DAProgress.select(
    'sourceId',
    'DAProgress'
)

#display(DAProgress)

#---
# modality selection

# no data on this yet
ModalitySelection = spark.sql(f"""
    SELECT 
    distinct
    e.payload.sourceId,
    null as PostUnderstanding,
    null as ModalityLocation,
    null as ModalityType,
    null as PostTransplant,
    null as PostConservativeCare,
    null as PostClearMatterMost,
    null as PostDASupport

    -- e.payload.typeformAnswersInput.title,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_rate_understanding`.input_text as PostUnderstanding,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_treatment_location_preference`.poll_option_content[0] as ModalityLocation,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_treatment_preference_dialysis_type`.poll_option_content[0] as ModalityType,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_open_to_transplant`.poll_option_content as PostTransplant,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_open_to_conservative_care`.poll_option_content as PostConservativeCare,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Clear_matters_most`.poll_option_content[0] as PostClearMatterMost,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Decision_support`.poll_option_content[0] as PostDASupport
    FROM PRODEU.events e
    WHERE e.type = 'typeform-responses-sent'  
    --AND e.payload.typeformAnswersInput.title = 'Treatment Preference'
            
""")
#display(ModalitySelection)

activated_program_events = spark.sql(f"""
    SELECT
    DISTINCT e.payload.clientId as sourceId,
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-sent') AS Total_program_task_sent, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-sent' and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_sent_last7Days, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-skipped') AS Total_program_task_skipped,
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-skipped'  and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_skipped_last7Days, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-completed') AS Total_program_task_completed, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-completed'  and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_completed_last7Days, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-continued') AS Total_program_task_continued,
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-continued'  and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_continued_last7Days
    FROM prodeu.events e 
    JOIN prodeu.patients p on e.payload.clientId = p.clientId
    WHERE p.clinicId in ('83a53614-f7cc-4ebf-a117-3beceda02747','fff0ed73-282e-4f22-a7d9-e9396aa0af99','62ca07e4-b021-4324-a4ca-35f95831c281','152aa029-a643-4ee8-b5c6-246294603453') 
    GROUP BY e.payload.clientId                               
                                 
""")

# join program progress and modality selection
videoask_onboarding = videoask_onboarding.join(program_progress, on = 'sourceId', how = 'left')
videoask_onboarding = videoask_onboarding.join(ModalitySelection, on = 'sourceId', how = 'left')
videoask_onboarding = videoask_onboarding.join(DAProgress, on = 'sourceId', how = 'left')
videoask_onboarding = videoask_onboarding.join(activated_program_events, on = 'sourceId', how = 'left')

videoask_onboarding = videoask_onboarding.select(
    'sourceId',
    'Clinic',
    'ProgramProgress',
    'DAprogress',
    'InitialLocation',
    'InitialDialysisType',
    'InitialConfidence',
    'InitialCKDUnderstanding',
    'InitialTransplant',
    'PostUnderstanding',
    'ModalityLocation',
    'ModalityType',
    'PostTransplant',
    'PostConservativeCare',
    'PostClearMatterMost',
    'PostDASupport',
    'ProgramDays',
    'Comms_Pref',
    'Call',
    'Total_program_task_sent',
    'Program_task_sent_last7Days',
    'Total_program_task_skipped',
    'Program_task_skipped_last7Days',
    'Total_program_task_completed',
    'Program_task_completed_last7Days',
    'Total_program_task_continued',
    'Program_task_continued_last7Days'
)


display(videoask_onboarding)
#display(program_progress)

sourceId,Clinic,ProgramProgress,DAprogress,InitialLocation,InitialDialysisType,InitialConfidence,InitialCKDUnderstanding,InitialTransplant,PostUnderstanding,ModalityLocation,ModalityType,PostTransplant,PostConservativeCare,PostClearMatterMost,PostDASupport,ProgramDays,Comms_Pref,Call,Total_program_task_sent,Program_task_sent_last7Days,Total_program_task_skipped,Program_task_skipped_last7Days,Total_program_task_completed,Program_task_completed_last7Days,Total_program_task_continued,Program_task_continued_last7Days
19d528cc-9354-4875-b754-e9d1b32c5cfb,Clinic N,5.88,20.0%,Home dialysis,Tube in my belly (Peritoneal Dialysis),6,8,Yes,null,null,null,null,null,null,null,Monday,SMS/Text,Get started without a call,8,1,3,1,1,0,0,0
662bae46-86fc-4b43-bcd2-8d009e8f2966,Clinic C,29.41,40.0%,Home dialysis,Needle access in my arm (Hemodialysis),8,9,No,null,null,null,null,null,null,null,Monday,SMS/Text,Get started without a call,4,1,1,0,4,1,4,2
cd6244bb-ee8c-40f8-8599-57194f42dd18,Clinic N,23.53,40.0%,In-center dialysis,Needle access in my arm (Hemodialysis),9,5,Yes,null,null,null,null,null,null,null,Wednesday,SMS/Text,Get started without a call,9,1,2,0,3,0,2,0
446ea9f9-cc0c-4c05-a1e2-96c612955c78,Clinic N,23.53,40.0%,Home dialysis,Needle access in my arm (Hemodialysis),5,8,No,null,null,null,null,null,null,null,Thursday,SMS/Text,Get started without a call,10,1,3,1,3,1,1,1
42e0c7e1-fba3-4978-a7fa-862ad9bc4738,Clinic U,5.88,20.0%,Home dialysis,Needle access in my arm (Hemodialysis),6,7,Yes,null,null,null,null,null,null,null,Monday,Email,Get started without a call,2,1,0,0,1,0,0,0
98e33549-769c-4d4f-a74b-e202ec754590,Clinic U,5.88,20.0%,Home dialysis,Needle access in my arm (Hemodialysis),0,1,No,null,null,null,null,null,null,null,Monday,Email,Get started without a call,2,1,0,0,1,0,0,0
43881376-f4fe-4789-a19a-bdfd550bd916,Clinic N,5.88,20.0%,In-center dialysis,Tube in my belly (Peritoneal Dialysis),10,6,No,null,null,null,null,null,null,null,Tuesday,SMS/Text,Get started without a call,1,1,0,0,1,1,0,0
ea836c41-5a72-4bd8-b11c-43a5516eac5f,Clinic B,88.24,80.0%,In-center dialysis,Needle access in my arm (Hemodialysis),10,4,Yes,null,null,null,null,null,null,null,Monday,SMS/Text,Book a call,16,1,0,0,19,1,12,0
2f20e3c7-2f37-4686-9016-550784502cbe,Clinic B,52.94,40.0%,Home dialysis,Needle access in my arm (Hemodialysis),5,5,No,null,null,null,null,null,null,null,Thursday,Email,Get started without a call,21,2,5,0,7,0,6,0


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# ea836c41-5a72-4bd8-b11c-43a5516eac5f



In [0]:
if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"


if selected_activated_patient == "Unselected":
    # Set the query condition to an empty string or a condition that always evaluates to true
    activated_query_condition = ""
else:
    # Use the selected patient name in your query condition
    activated_query_condition = f" and e.payload.sourceId = '{selected_activated_patient}'"


activated_program_events = spark.sql(f"""
    SELECT
    DISTINCT e.payload.clientId,
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-sent') AS Total_program_task_sent, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-sent' and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_sent_last7Days, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-skipped') AS Total_program_task_skipped,
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-skipped'  and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_skipped_last7Days, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-completed') AS Total_program_task_completed, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-completed'  and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_completed_last7Days, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-continued') AS Total_program_task_continued,
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-continued'  and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_continued_last7Days
    FROM prodeu.events e 
    JOIN prodeu.patients p on e.payload.clientId = p.clientId
    WHERE p.clinicId in ('83a53614-f7cc-4ebf-a117-3beceda02747','fff0ed73-282e-4f22-a7d9-e9396aa0af99','62ca07e4-b021-4324-a4ca-35f95831c281','152aa029-a643-4ee8-b5c6-246294603453') 
    and {query_condition}
   {activated_query_condition}  
    GROUP BY e.payload.clientId                               
                                 
""")
display(activated_program_events)

clientId,Total_program_task_sent,Program_task_sent_last7Days,Total_program_task_skipped,Program_task_skipped_last7Days,Total_program_task_completed,Program_task_completed_last7Days,Total_program_task_continued,Program_task_continued_last7Days
662bae46-86fc-4b43-bcd2-8d009e8f2966,4,1,1,0,4,1,4,2
cd6244bb-ee8c-40f8-8599-57194f42dd18,7,1,2,0,3,0,2,0
2f20e3c7-2f37-4686-9016-550784502cbe,13,1,5,0,7,0,6,0
19d528cc-9354-4875-b754-e9d1b32c5cfb,7,1,3,1,1,0,0,0
ea836c41-5a72-4bd8-b11c-43a5516eac5f,10,1,0,0,19,1,12,0
446ea9f9-cc0c-4c05-a1e2-96c612955c78,7,1,3,1,3,1,1,1
43881376-f4fe-4789-a19a-bdfd550bd916,1,1,0,0,1,1,0,0
42e0c7e1-fba3-4978-a7fa-862ad9bc4738,2,1,0,0,1,0,0,0
98e33549-769c-4d4f-a74b-e202ec754590,2,1,0,0,1,0,0,0


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# program task sent, skipped, completed, continued
# list of activated patients

activated_program_events = spark.sql(f"""
    SELECT
    DISTINCT e.payload.clientId,
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-sent') AS Total_program_task_sent, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-sent' and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_sent_last7Days, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-skipped') AS Total_program_task_skipped,
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-skipped'  and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_skipped_last7Days, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-completed') AS Total_program_task_completed, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-completed'  and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_completed_last7Days, 
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-continued') AS Total_program_task_continued,
    COUNT(e.payload.clientId) FILTER(WHERE e.type = 'program-task-continued'  and e.timestamp >= '{start_date_last_7_days}' AND e.timestamp <= '{end_date_last_7_days}') AS Program_task_continued_last7Days
    FROM prodeu.events e 
    JOIN prodeu.patients p on e.payload.clientId = p.clientId
    WHERE p.clinicId in ('83a53614-f7cc-4ebf-a117-3beceda02747','fff0ed73-282e-4f22-a7d9-e9396aa0af99','62ca07e4-b021-4324-a4ca-35f95831c281','152aa029-a643-4ee8-b5c6-246294603453') 
    --{activated_query_condition}
    GROUP BY e.payload.clientId                               
                                 
""")
display(activated_program_events)

clientId,Total_program_task_sent,Program_task_sent_last7Days,Total_program_task_skipped,Program_task_skipped_last7Days,Total_program_task_completed,Program_task_completed_last7Days,Total_program_task_continued,Program_task_continued_last7Days
63873225-73ac-484a-a805-c9a8208d1548,0,0,0,0,0,0,0,0
886d657f-01ea-45c1-86dc-af2a3d955d92,0,0,0,0,0,0,0,0
0c1071e6-6c91-4b1d-8601-968b5919264a,0,0,0,0,0,0,0,0
76acf550-dbe7-4606-bca2-619cc9cc0ad3,12,0,1,0,10,0,6,0
4af046b7-5e2e-47c4-a0b6-18867740c022,11,1,2,0,5,3,3,3
b05469c2-a2f7-4cf4-b58e-cfa37f4c76ec,0,0,0,0,0,0,0,0
a8115fc3-d97f-454c-944d-d282711ecec4,14,1,1,0,9,1,12,5
64e8d75b-57c5-446a-abc2-cc59281fbedd,9,1,0,0,8,0,3,0
e5347001-78b1-4a51-bd39-86ab77182a94,11,0,3,0,3,0,2,0
92f08ffa-3b36-4c2b-b72a-23709f2747d3,0,0,0,0,0,0,0,0


In [0]:
if activated_list:
    # If not empty, include the IN clause in the SQL query
    if len(activated_list) == 1:
        query_condition = f"e.payload.sourceId = '{activated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(activated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"


if selected_activated_patient == "Unselected":
    # Set the query condition to an empty string or a condition that always evaluates to true
    activated_query_condition = ""
else:
    # Use the selected patient name in your query condition
    activated_query_condition = f" and e.payload.sourceId = '{selected_activated_patient}'"



# program task sent and program task sent last week?
program_task = spark.sql(f"""
   SELECT
   date(e.timestamp) as Date,
   e.payload.clientId,
   --p.clinicId as Clinic,
   --count(e.payload.clientId),
   count(e.payload.clientId) FILTER(WHERE e.type = 'program-task-sent') as Numb_task_sent,
   count(e.payload.clientId) FILTER(WHERE e.type = 'program-task-completed') as Numb_task_completed,
   count(e.payload.clientId) FILTER(WHERE e.type = 'program-task-skipped') as Numb_task_skipped,
   count(e.payload.clientId) FILTER(WHERE e.type = 'program-task-continued') as Numb_task_continued
   FROM prodeu.events e
   JOIN prodeu.patients p on e.payload.clientId = p.clientId
   
   WHERE p.clinicId in ('83a53614-f7cc-4ebf-a117-3beceda02747','fff0ed73-282e-4f22-a7d9-e9396aa0af99','62ca07e4-b021-4324-a4ca-35f95831c281','152aa029-a643-4ee8-b5c6-246294603453')
   and {query_condition}
   {activated_query_condition}  
   GROUP BY Date,  e.payload.clientId                     
""")
display(program_task)

Date,clientId,Numb_task_sent,Numb_task_completed,Numb_task_skipped,Numb_task_continued
2023-11-22,2f20e3c7-2f37-4686-9016-550784502cbe,1,0,0,0
2023-11-22,662bae46-86fc-4b43-bcd2-8d009e8f2966,0,0,0,0
2023-12-11,42e0c7e1-fba3-4978-a7fa-862ad9bc4738,1,0,0,0
2023-12-01,98e33549-769c-4d4f-a74b-e202ec754590,0,1,0,0
2023-09-16,ea836c41-5a72-4bd8-b11c-43a5516eac5f,0,0,0,0
2023-12-06,cd6244bb-ee8c-40f8-8599-57194f42dd18,1,0,0,0
2023-12-12,43881376-f4fe-4789-a19a-bdfd550bd916,1,0,0,0
2023-11-24,2f20e3c7-2f37-4686-9016-550784502cbe,0,0,0,0
2023-11-27,42e0c7e1-fba3-4978-a7fa-862ad9bc4738,0,1,0,0
2023-12-04,ea836c41-5a72-4bd8-b11c-43a5516eac5f,1,0,0,0


Databricks visualization. Run in Databricks to view.

In [0]:
if inactivated_list:
    # If not empty, include the IN clause in the SQL query
    if len(inactivated_list) == 1:
        query_condition = f"e.payload.sourceId = '{inactivated_list[0]}'"
    else:
        query_condition = f"e.payload.sourceId IN {format(tuple(inactivated_list))}"
else:
    # If empty, set a condition that is always false so that the query doesn't return any results
    query_condition = "1 = 0"


if selected_inactivated_patient == "Unselected":
    # Set the query condition to an empty string or a condition that always evaluates to true
    inactivated_query_condition = ""
else:
    # Use the selected patient name in your query condition
    inactivated_query_condition = f" and e.payload.sourceId = '{selected_inactivated_patient}'"

Inactivated_individual_patient_metrics = spark.sql(f"""
       SELECT
       date(e.timestamp) as Date,
       e.payload.clientId as InactivatedPatient,
       any_value(CASE 
        WHEN p.clinicId = "83a53614-f7cc-4ebf-a117-3beceda02747" THEN "Balboa"
        WHEN p.clinicId = "fff0ed73-282e-4f22-a7d9-e9396aa0af99" THEN "Neph Associates"
        WHEN p.clinicId = '62ca07e4-b021-4324-a4ca-35f95831c281' THEN 'D2C US'
        WHEN p.clinicId = '152aa029-a643-4ee8-b5c6-246294603453' THEN 'D2C Canada'
        END) as Clinic,
       count(e.payload.clientId) filter(where e.type = 'iterable-emailSend') NumEmailSent,
       count(e.payload.clientId) filter(where e.type = 'iterable-smsSend') as NumSMSSent,
       count(e.payload.clientId) filter(where e.type = 'iterable-emailOpen') as NumEmailOpen,
       count(e.payload.clientId) filter(where e.type = 'iterable-smsClick') as NumsmsClick
       FROM prodeu.events e
       LEFT JOIN prodeu.patients p on e.payload.sourceId = p.clientId
       WHERE p.clinicId in ('83a53614-f7cc-4ebf-a117-3beceda02747','fff0ed73-282e-4f22-a7d9-e9396aa0af99','62ca07e4-b021-4324-a4ca-35f95831c281','152aa029-a643-4ee8-b5c6-246294603453')
       and {query_condition}
      {inactivated_query_condition} 
       GROUP BY date, e.payload.clientId 
    """)
display(Inactivated_individual_patient_metrics)

Date,InactivatedPatient,Clinic,NumEmailSent,NumSMSSent,NumEmailOpen,NumsmsClick
2023-10-06,b4b9e2e8-6966-48fc-821f-086000c82eff,Neph Associates,0,0,0,0
2023-10-09,b4b9e2e8-6966-48fc-821f-086000c82eff,Neph Associates,5,4,1,0
2023-10-10,b4b9e2e8-6966-48fc-821f-086000c82eff,Neph Associates,0,0,4,0
2023-10-11,b4b9e2e8-6966-48fc-821f-086000c82eff,Neph Associates,5,4,2,0
2023-10-11,caf94a40-fd7a-4a50-b97a-86db95d707f5,Neph Associates,0,0,0,0
2023-10-11,d0efe43f-e990-4b8d-ae2d-b49950461d37,Neph Associates,0,0,0,0
2023-10-12,b4b9e2e8-6966-48fc-821f-086000c82eff,Neph Associates,0,0,3,0
2023-10-12,caf94a40-fd7a-4a50-b97a-86db95d707f5,Neph Associates,4,2,0,0
2023-10-12,d0efe43f-e990-4b8d-ae2d-b49950461d37,Neph Associates,5,3,5,0
2023-10-13,b4b9e2e8-6966-48fc-821f-086000c82eff,Neph Associates,0,2,2,0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select e.payload.iterableEvent.datafields.messageId 
from prodeu.events e
where e.type = 'iterable-smsClick'

messageId
7398ecf8ee8040d5bca773e35de7eb34
7398ecf8ee8040d5bca773e35de7eb34
380d5ede0d884fea885b6788c32c7bba
e1a2c4b3f73d472197dfe07c54fd4acd
e1a2c4b3f73d472197dfe07c54fd4acd
e1a2c4b3f73d472197dfe07c54fd4acd
0ee43c5ae50248229e65fc56722cc156
e1a2c4b3f73d472197dfe07c54fd4acd
789f277ce4404ee991cb459ebe48178f
789f277ce4404ee991cb459ebe48178f


In [0]:
# program task skipped, not actioned in two weeks
program_task_skipped = spark.sql(f"""
   SELECT
   e.payload.sourceid,
   e.payload.clientId,
   e.timestamp
   FROM prodeu.events e
   JOIN prodeu.patients p on e.payload.clientId = p.clientId
   WHERE e.type = 'program-task-skipped' 
   AND p.clinicId in ('83a53614-f7cc-4ebf-a117-3beceda02747','fff0ed73-282e-4f22-a7d9-e9396aa0af99','62ca07e4-b021-4324-a4ca-35f95831c281','152aa029-a643-4ee8-b5c6-246294603453')                         
""")
display(program_task_skipped)

sourceid,clientId,timestamp
c5a3b526-acfb-4543-ace6-171f4e999ba1,c5a3b526-acfb-4543-ace6-171f4e999ba1,2023-11-17T08:00:21.377Z
e5347001-78b1-4a51-bd39-86ab77182a94,e5347001-78b1-4a51-bd39-86ab77182a94,2023-11-20T08:00:21.862Z
a8115fc3-d97f-454c-944d-d282711ecec4,a8115fc3-d97f-454c-944d-d282711ecec4,2023-11-21T08:00:19.459Z
42e0b26f-a4c4-4761-b4e7-37db17ec5020,42e0b26f-a4c4-4761-b4e7-37db17ec5020,2023-11-21T08:00:23.464Z
d439f06f-7cbe-4670-a3a9-561855a5eaa8,d439f06f-7cbe-4670-a3a9-561855a5eaa8,2023-11-22T08:00:21.023Z
2f20e3c7-2f37-4686-9016-550784502cbe,2f20e3c7-2f37-4686-9016-550784502cbe,2023-11-23T08:00:23.579Z
19d528cc-9354-4875-b754-e9d1b32c5cfb,19d528cc-9354-4875-b754-e9d1b32c5cfb,2023-11-27T08:00:22.117Z
cd6244bb-ee8c-40f8-8599-57194f42dd18,cd6244bb-ee8c-40f8-8599-57194f42dd18,2023-11-29T08:00:21.884Z
446ea9f9-cc0c-4c05-a1e2-96c612955c78,446ea9f9-cc0c-4c05-a1e2-96c612955c78,2023-11-30T08:00:20.529Z
662bae46-86fc-4b43-bcd2-8d009e8f2966,662bae46-86fc-4b43-bcd2-8d009e8f2966,2023-12-04T08:00:21.339Z


In [0]:
# program task-completed total, and last week
program_task_completed = spark.sql(f"""
   SELECT
   e.payload.sourceid,
   e.payload.clientId,
   e.timestamp
   FROM prodeu.events e
   WHERE e.type = 'program-task-completed'                           
""")
display(program_task_completed)

sourceid,clientId,timestamp
76acf550-dbe7-4606-bca2-619cc9cc0ad3,76acf550-dbe7-4606-bca2-619cc9cc0ad3,2023-09-29T18:23:04.629Z
ea836c41-5a72-4bd8-b11c-43a5516eac5f,ea836c41-5a72-4bd8-b11c-43a5516eac5f,2023-10-02T17:01:27.280Z
ea836c41-5a72-4bd8-b11c-43a5516eac5f,ea836c41-5a72-4bd8-b11c-43a5516eac5f,2023-10-02T17:04:06.626Z
ea836c41-5a72-4bd8-b11c-43a5516eac5f,ea836c41-5a72-4bd8-b11c-43a5516eac5f,2023-10-02T17:09:37.457Z
e5347001-78b1-4a51-bd39-86ab77182a94,e5347001-78b1-4a51-bd39-86ab77182a94,2023-10-03T16:57:46.652Z
a8115fc3-d97f-454c-944d-d282711ecec4,a8115fc3-d97f-454c-944d-d282711ecec4,2023-10-04T02:29:40.894Z
42e0b26f-a4c4-4761-b4e7-37db17ec5020,42e0b26f-a4c4-4761-b4e7-37db17ec5020,2023-10-04T22:18:02.157Z
76acf550-dbe7-4606-bca2-619cc9cc0ad3,76acf550-dbe7-4606-bca2-619cc9cc0ad3,2023-11-17T16:18:46.109Z
76acf550-dbe7-4606-bca2-619cc9cc0ad3,76acf550-dbe7-4606-bca2-619cc9cc0ad3,2023-11-17T17:17:06.076Z
2f20e3c7-2f37-4686-9016-550784502cbe,2f20e3c7-2f37-4686-9016-550784502cbe,2023-11-17T21:02:56.860Z


In [0]:
# program task continued
program_task_continued = spark.sql(f"""
   SELECT
   e.payload.sourceid,
   e.payload.clientId,
   e.timestamp
   FROM prodeu.events e
   WHERE e.type = 'program-task-continued'                           
""")
display(program_task_continued)

sourceid,clientId,timestamp
ea836c41-5a72-4bd8-b11c-43a5516eac5f,ea836c41-5a72-4bd8-b11c-43a5516eac5f,2023-10-02T17:00:17.068Z
ea836c41-5a72-4bd8-b11c-43a5516eac5f,ea836c41-5a72-4bd8-b11c-43a5516eac5f,2023-10-02T17:04:06.790Z
ea836c41-5a72-4bd8-b11c-43a5516eac5f,ea836c41-5a72-4bd8-b11c-43a5516eac5f,2023-10-02T17:08:18.209Z
76acf550-dbe7-4606-bca2-619cc9cc0ad3,76acf550-dbe7-4606-bca2-619cc9cc0ad3,2023-11-17T16:18:43.935Z
76acf550-dbe7-4606-bca2-619cc9cc0ad3,76acf550-dbe7-4606-bca2-619cc9cc0ad3,2023-11-17T17:14:22.296Z
2f20e3c7-2f37-4686-9016-550784502cbe,2f20e3c7-2f37-4686-9016-550784502cbe,2023-11-17T21:02:55.027Z
662bae46-86fc-4b43-bcd2-8d009e8f2966,662bae46-86fc-4b43-bcd2-8d009e8f2966,2023-11-20T22:36:23.740Z
662bae46-86fc-4b43-bcd2-8d009e8f2966,662bae46-86fc-4b43-bcd2-8d009e8f2966,2023-11-20T22:39:28.063Z
a8115fc3-d97f-454c-944d-d282711ecec4,a8115fc3-d97f-454c-944d-d282711ecec4,2023-11-22T01:00:07.031Z
a8115fc3-d97f-454c-944d-d282711ecec4,a8115fc3-d97f-454c-944d-d282711ecec4,2023-11-22T01:05:24.076Z


In [0]:
average_initial_confidence = videoask_onboarding.select(
    F.round(F.avg("InitialConfidence"), 2).alias("AverageInitialConfidence"))
display(average_initial_confidence)

average_initial_understanding = videoask_onboarding.select(
    F.round(F.avg("InitialCKDUnderstanding"), 2).alias("AverageInitialCKDUnderstanding"))
display(average_initial_understanding)



AverageInitialConfidence
6.56


Databricks visualization. Run in Databricks to view.

AverageInitialCKDUnderstanding
5.89


Databricks visualization. Run in Databricks to view.

In [0]:
display(videoask_onboarding)

sourceId,Clinic,ProgramProgress,DAprogress,InitialLocation,InitialDialysisType,InitialConfidence,InitialCKDUnderstanding,InitialTransplant,PostUnderstanding,ModalityLocation,ModalityType,PostTransplant,PostConservativeCare,PostClearMatterMost,PostDASupport,ProgramDays,Comms_Pref,Call,Total_program_task_sent,Program_task_sent_last7Days,Total_program_task_skipped,Program_task_skipped_last7Days,Total_program_task_completed,Program_task_completed_last7Days,Total_program_task_continued,Program_task_continued_last7Days
42e0c7e1-fba3-4978-a7fa-862ad9bc4738,D2C US,5.88,20.0%,Home dialysis,Needle access in my arm (Hemodialysis),6,7,Yes,null,null,null,null,null,null,null,Monday,Email,Get started without a call,2,1,0,0,1,0,0,0
43881376-f4fe-4789-a19a-bdfd550bd916,Neph Associates,5.88,20.0%,In-center dialysis,Tube in my belly (Peritoneal Dialysis),10,6,No,null,null,null,null,null,null,null,Tuesday,SMS/Text,Get started without a call,1,1,0,0,1,1,0,0
2f20e3c7-2f37-4686-9016-550784502cbe,Balboa,52.94,40.0%,Home dialysis,Needle access in my arm (Hemodialysis),5,5,No,null,null,null,null,null,null,null,Thursday,Email,Get started without a call,21,2,5,0,7,0,6,0
98e33549-769c-4d4f-a74b-e202ec754590,D2C US,5.88,20.0%,Home dialysis,Needle access in my arm (Hemodialysis),0,1,No,null,null,null,null,null,null,null,Monday,Email,Get started without a call,2,1,0,0,1,0,0,0
ea836c41-5a72-4bd8-b11c-43a5516eac5f,Balboa,88.24,80.0%,In-center dialysis,Needle access in my arm (Hemodialysis),10,4,Yes,null,null,null,null,null,null,null,Monday,SMS/Text,Book a call,16,1,0,0,19,1,12,0
19d528cc-9354-4875-b754-e9d1b32c5cfb,Neph Associates,5.88,20.0%,Home dialysis,Tube in my belly (Peritoneal Dialysis),6,8,Yes,null,null,null,null,null,null,null,Monday,SMS/Text,Get started without a call,8,1,3,1,1,0,0,0
446ea9f9-cc0c-4c05-a1e2-96c612955c78,Neph Associates,23.53,40.0%,Home dialysis,Needle access in my arm (Hemodialysis),5,8,No,null,null,null,null,null,null,null,Thursday,SMS/Text,Get started without a call,10,1,3,1,3,1,1,1
cd6244bb-ee8c-40f8-8599-57194f42dd18,Neph Associates,23.53,40.0%,In-center dialysis,Needle access in my arm (Hemodialysis),9,5,Yes,null,null,null,null,null,null,null,Wednesday,SMS/Text,Get started without a call,9,1,2,0,3,0,2,0
662bae46-86fc-4b43-bcd2-8d009e8f2966,D2C Canada,29.41,40.0%,Home dialysis,Needle access in my arm (Hemodialysis),8,9,No,null,null,null,null,null,null,null,Monday,SMS/Text,Get started without a call,4,1,1,0,4,1,4,2


In [0]:
%sql
select * 
from prodeu.events e
where e.type = 'videoask-responses-sent'
and e.payload.sourceId = '662bae46-86fc-4b43-bcd2-8d009e8f2966'

payload regulatoryRegion tenant timestamp type List(null, null, null, null, null, null, null, 662bae46-86fc-4b43-bcd2-8d009e8f2966, null, null, null, null, null, null, null, 493f8fe2-5cae-45e0-9163-e2496985a5d2, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, bfd4bd77-757d-4f44-9fff-a423d2559275, null, null, null, List(null, null, videoask), null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, canada, null, null, null, null, null, null, 493f8fe2-5cae-45e0-9163-e2496985a5d2, 662bae46-86fc-4b43-bcd2-8d009e8f2966, null, null, null, null, null, null, null, null, null, 4, null, null, videoask-responses-sent, null, null, null, null, null, null, null, List(fbcvzhxvu, List(null, null, null, null, null, null, null, null, null, null, null, List(64f519c6-2a7e-4a88-be05-1b282bfe0beb, 👌 I already knew this!, 5c903fb9-50dc-4ed3-ab95-f332fe218cf7, List(List(👌 I already knew this!, 5c903fb9-50dc-4ed3-ab95-f332fe218cf7, 448cb7b2-7c60-48e9-8f52-31cb9d839982)), 344e277c-8a24-4115-876f-993f9b30da6b, poll), List(d908e33c-86e1-47d3-b1b7-b63af610df19, Get started, ca1c0701-81f6-4219-a440-92bf24da1aec, List(List(Get started, ca1c0701-81f6-4219-a440-92bf24da1aec, a0fe7bc0-09fe-46b7-9003-936cb96cee42)), 0a87b6cd-3136-4ab9-bf6c-631d7e706307, poll), null, List(21720a16-fe2b-4839-9192-5827dfcc1d00, Filtering waste from the blood and regulating fluid balance, 30affd36-eb0e-4668-b5c7-bd9d09aec93e, List(List(Filtering waste from the blood and regulating fluid balance, 30affd36-eb0e-4668-b5c7-bd9d09aec93e, 0e401cd2-478f-4c88-87af-8a310468b12d)), b1e80be5-d63c-49a2-91ed-6baaf71ee6a3, poll), null, null, List(378ade8a-040f-405d-8e8d-9b3f158a3fd3, You can reduce the fear and anxiety that may come along with it., c3d067ea-4033-4046-907d-f3f288435380, List(List(You can reduce the fear and anxiety that may come along with it., c3d067ea-4033-4046-907d-f3f288435380, 5f21cddc-3cc2-4ded-9d0c-53e89219c7b3)), 8d98002f-2948-4142-84f6-cade777ac3ab, poll), null, null, null, null, null, null, null, null, null, null, null, null, null, List(3171d61e-6384-4f03-95fb-6e1ed48564c1, I want to keep going!, f45cd26b-777c-4792-bd8b-25a1d424be25, List(List(I want to keep going!, f45cd26b-777c-4792-bd8b-25a1d424be25, a4b30966-dd01-4ce6-91cc-bfe2d0b9a31f)), da83c41f-3dd4-4fdb-a0a9-cd2c221755b5, poll), null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, List(240e02db-2c26-4637-9530-214530ad5700, Nothing yet, I will let you know if there is anything, 51fe0c16-f914-4c31-b99b-789cd38a4aed, List(List(Nothing yet, I will let you know if there is anything, 51fe0c16-f914-4c31-b99b-789cd38a4aed, 542cbcca-dc5c-4841-9c51-149b0dff36fd)), 8cd87d72-242e-4bb5-acb1-65088175e624, poll), null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, List(5ca3fa85-83b1-48c7-8a58-0911b55e7b0d, Next: Test your knowledge, 93726285-655c-4b3f-9b1a-ab916180c600, List(List(Next: Test your knowledge, 93726285-655c-4b3f-9b1a-ab916180c600, a90e72ac-41cc-40f1-871b-1d1a63a4e9a4)), 8b225e3a-98a3-4739-9505-6d20e7986dbe, poll), null, null, null, null, null, null, null, null, null, List(aea4c3c5-b148-4e83-8018-b72f4fede5ff, Continue, 5b961e69-a5d8-4c6b-b406-76ae51d76631, List(List(Continue, 5b961e69-a5d8-4c6b-b406-76ae51d76631, 6c703c66-e0d1-47bf-a9f2-c3466220e75d)), 38530987-373d-483a-9c3e-ec5a0030d0c1, poll)), CKD & Kidney Basics), null, null, null, null) canada 4 2023-11-20T22:36:24.485Z videoask-responses-sent List(null, null, null, null, null, null, null, 662bae46-86fc-4b43-bcd2-8d009e8f2966, null, null, null, null, null, null, null, 493f8fe2-5cae-45e0-9163-e2496985a5d2, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null,

In [0]:
# sankey graph


In [0]:
ModalitySelection = spark.sql(f"""
    SELECT 
    distinct
    e.payload.sourceId,
    e.payload.typeformAnswersInput.title,
    e.payload.typeformAnswersInput.questionAnswers,
    e.payload.typeformAnswersInput.questionAnswers.`Q-Interest_connecting_others`,
    e.payload.typeformAnswersInput.questionAnswers.`Q-Top_3_values`,
    e.payload.typeformAnswersInput.questionAnswers.`Q-Main_transportation`
    --e.payload.typeformAnswersInput.questionAnswers.`Q-Decision_support`
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_rate_understanding`.input_text as PostUnderstanding,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_treatment_location_preference`.poll_option_content[0] as ModalityLocation,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_treatment_preference_dialysis_type`.poll_option_content[0] as ModalityType,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_open_to_transplant`.poll_option_content as PostTransplant,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Post_open_to_conservative_care`.poll_option_content as PostConservativeCare,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Clear_matters_most`.poll_option_content[0] as PostClearMatterMost,
    -- e.payload.typeformAnswersInput.questionAnswers.`Q-Decision_support`.poll_option_content[0] as PostDASupport
    FROM PRODEU.events e
    WHERE e.type = 'typeform-responses-sent'  
    AND e.payload.typeformAnswersInput.title != 'Call Log'
            
""")
display(ModalitySelection)

sourceId title questionAnswers Q-Interest_connecting_others Q-Top_3_values Q-Main_transportation e5347001-78b1-4a51-bd39-86ab77182a94 Profile List(null, null, null, null, List(List(BxDl5AqSvwJi, I like them), List(LkpvwovRWinP, 0fbaa2aa-23de-4868-88cc-76464727987e, multiple_choice), choice), null, null, null, null, null, null, null, null, null, null, null, List(List(MiDQmxO7uOUC, Not really), List(eocMIYflUwUR, a801c172-b9a3-4990-ab9e-7d032790e569, multiple_choice), choice), null, null, List(List(3Ednm06mbv3j, Q-Beacon_statement, short_text), Lowering my A1C levels, text), null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, List(List(to4WJ9Xc7K55, Employed, working 40 or more hours per week, f2190325-cb34-4b14-b724-1581a7caa7a4), List(H1RuDnLR5yLx, Q-Employment, multiple_choice), choice), null, List(List(List(CwL6wMEHDX3c, d1eM89Hsnlk3), List(Nelson Mandela, Frida), null, List(ced3748f-e36e-4f7a-98c0-99c35335c663, 4e661d3f-3320-46e0-beae-20554ffdf5b6)), List(hERjGAhS5QHv, Q-Favorite_personality, picture_choice), choices), null, null, null, null, null, null, List(List(FZvWlhaKvsky, Q-Interest_connecting_others, opinion_scale), 3, number), List(List(eoj3euUqZ51p, Q-Interest_overcome_worries, opinion_scale), 3, number), List(List(ZJRvK4LpqpwG, Q-Interest_treat_options, opinion_scale), 5, number), List(List(JriT7dvy89S5, Q-Interest_treat_selection, opinion_scale), 5, number), List(List(CMMtEbMSAbaV, Q-Interest_understand_ckd, opinion_scale), 5, number), null, null, null, null, null, List(List(E888YV1a8jd5, Home-cooked by someone else, f1eb5333-a8e9-4ce6-929f-244c5324548c), List(hFz9OVaDbDkX, Q-Meals, multiple_choice), choice), null, null, null, null, null, null, List(false, List(bh8ecBloX6Zr, Q-Participated_forum, yes_no), boolean), null, null, null, null, null, null, null, List(List(iYoUAaxcg7wO, You've got it right!, 32f2c117-a49e-416c-a1b2-2e3a0aa24a3d), List(NAW4elJdVWMl, Q-Pref_name, multiple_choice), choice), null, null, null, null, null, null, null, null, null, List(List(boVgKiLrSQJD, A digital calendar or app, aab6296e-f8d5-4323-a6f0-987e40e3190e), List(DMljReu0KL2i, Q-Track_schedule, multiple_choice), choice), null, null, List(List(5UeD3SEkoBDm, Q-Upcoming_events, short_text), None, text), null, null, null, null, null, List(List(ILg7paGygJJG, Q-interest_diet_changes, opinion_scale), 5, number), null, null, null, null, null, List(List(sUutEcGaUXf2, I like them), List(q5fVPeP7df6f, 1f65085c-78a1-43e4-a423-11e02f6b15b1, multiple_choice), choice), null, null) List(List(FZvWlhaKvsky, Q-Interest_connecting_others, opinion_scale), 3, number) null null a8115fc3-d97f-454c-944d-d282711ecec4 Profile List(null, null, null, null, List(List(WgDX0Iu8nSdL, Not really), List(LkpvwovRWinP, 0fbaa2aa-23de-4868-88cc-76464727987e, multiple_choice), choice), null, null, null, null, null, null, null, null, null, null, null, List(List(ixObTheOqi8L, No preference), List(eocMIYflUwUR, a801c172-b9a3-4990-ab9e-7d032790e569, multiple_choice), choice), null, null, List(List(3Ednm06mbv3j, Q-Beacon_statement, short_text), I’ll be on cruise, text), null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, List(List(RpkoFkUEa6Tp, Retired, 9ffb6871-878a-4c23-b2be-28062dd3cde4), List(H1RuDnLR5yLx, Q-Employment, multiple_choice), choice), null, List(List(List(wRavWTh6cMPW, d1eM89Hsnlk3), List(Yoda, Frida), null, List(50158395-1245-4394-8579-5110c17a6e1f, 4e661d3f-3320-46e0-beae-20554ffdf5b6)), List(hERjGAhS5QHv, Q-Favorite_personality, picture_choice), choices), null, null, null, null, null, null, List(List(FZvWlhaKvsky, Q-Interest_connecting_others, opinion_scale), 1, number), List(List(eoj3euUqZ51p, Q-Interest_overcome_worries, opinion_scale), 2, number), List(List(ZJRvK4LpqpwG, Q-Interest_treat_options, opinion_scale), 2, number), List(List(JriT7dvy89S5, Q-Interest_treat_selection, opinion_scale), 5, number), List(List(CMMtEbMSAbaV, Q-Interest_understand_ckd, opinion_scale), 5, number), 

In [0]:
dialpad = spark.sql(f"""
   SELECT 
   distinct e.type
   --e.payload
   FROM prodeu.events e 
   where e.type like '%dialpad%'                           
""")
display(dialpad)

type


In [0]:
%sql
select * 
from prodeu.events e
where e.type = 'videoask-responses-sent'

payload regulatoryRegion tenant timestamp type List(null, null, null, null, null, null, null, a8115fc3-d97f-454c-944d-d282711ecec4, null, null, null, null, null, null, null, 0f06a8e1-c448-40e4-958e-69d25e64b5d4, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 8d0c7d08-ab54-4800-a095-a4a7e719ad40, null, null, null, List(null, null, videoask), null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, united-states, null, null, null, null, null, null, 0f06a8e1-c448-40e4-958e-69d25e64b5d4, a8115fc3-d97f-454c-944d-d282711ecec4, null, null, null, null, null, null, null, null, null, 4, null, null, videoask-responses-sent, null, null, null, null, null, null, null, List(ac5c2c13-c35f-442e-ab73-c6bb9c372d05, List(null, null, null, null, null, null, null, null, null, null, null, List(a272dcbf-c3c7-49e3-8f14-28a9155cad54, 🙌 This was helpful, 3ffafb02-1242-4b24-bba1-f91e736a3ee5, List(List(🙌 This was helpful, 3ffafb02-1242-4b24-bba1-f91e736a3ee5, ff0549ce-ee05-477c-b3ac-be987a845ed4)), 344e277c-8a24-4115-876f-993f9b30da6b, poll), List(480e069f-b57a-422d-a586-3214044e4284, Get started, ca1c0701-81f6-4219-a440-92bf24da1aec, List(List(Get started, ca1c0701-81f6-4219-a440-92bf24da1aec, a0fe7bc0-09fe-46b7-9003-936cb96cee42)), 0a87b6cd-3136-4ab9-bf6c-631d7e706307, poll), null, List(ed182009-a0fa-4657-8587-3ea187a08dfa, Filtering waste from the blood and regulating fluid balance, 30affd36-eb0e-4668-b5c7-bd9d09aec93e, List(List(Filtering waste from the blood and regulating fluid balance, 30affd36-eb0e-4668-b5c7-bd9d09aec93e, 0e401cd2-478f-4c88-87af-8a310468b12d)), b1e80be5-d63c-49a2-91ed-6baaf71ee6a3, poll), null, null, List(58248c7e-63a7-497c-92b0-e47c0913fe51, You can reduce the fear and anxiety that may come along with it., c3d067ea-4033-4046-907d-f3f288435380, List(List(You can reduce the fear and anxiety that may come along with it., c3d067ea-4033-4046-907d-f3f288435380, 5f21cddc-3cc2-4ded-9d0c-53e89219c7b3)), 8d98002f-2948-4142-84f6-cade777ac3ab, poll), null, null, null, null, null, null, null, null, null, null, null, null, null, List(5110cf70-15bf-4c78-90d7-06ca324855e5, I'm done for today!, b7013d8c-16e6-4eeb-a2a7-98002e47bcc5, List(List(I'm done for today!, b7013d8c-16e6-4eeb-a2a7-98002e47bcc5, 793e5b0a-7a4c-4f9b-9bff-08a3369e1d61)), da83c41f-3dd4-4fdb-a0a9-cd2c221755b5, poll), null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, List(5dc69bdf-cdde-4813-a174-9b495108d4a9, Learn more about the scientific study from the video, e4fbbc53-d520-4395-96af-2119c2e21c36, List(List(Learn more about the scientific study from the video, e4fbbc53-d520-4395-96af-2119c2e21c36, 4a47ebe5-83a7-4325-b071-a1c2eaae5d25), List(Review and select questions to ask your doctor from a list of common questions other patients have had, 308243f1-8449-429a-9398-8711996e306b, 5cbcbe1c-b919-4f21-9ed3-b70a8566776f)), 8cd87d72-242e-4bb5-acb1-65088175e624, poll), null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, List(2d5ad154-3969-4d88-86c6-76755497775d, Next: Test your knowledge, 93726285-655c-4b3f-9b1a-ab916180c600, List(List(Next: Test your knowledge, 93726285-655c-4b3f-9b1a-ab916180c600, a90e72ac-41cc-40f1-871b-1d1a63a4e9a4)), 8b225e3a-98a3-4739-9505-6d20e7986dbe, poll), null, null, null, null, null, null, null, null, null, List(4ccf5a2a-a917-4c96-9f7c-4c2c828c1342, Continue, 5b961e69-a5d8-4c6b-b406-76ae51d76631, List(List(Continue, 5b961e69-a5d8-4c6b-b406-76ae51d76631, 6c703c66-e0d1-47bf-a9f2-c3466220e75d)), 38530987-373d-483a-9c3e-ec5a0030d0c1, poll)), CKD & Kidney Basics), null, null, null, null) united-states 4 2023-08-22T20:28:21.094Z videoask-responses-sent List(null, null, null, null, null, null, 